<a href="https://colab.research.google.com/github/alozy15/A_B-Testing/blob/master/RV_M%26C_P%26L.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install pmdarima

In [ ]:
#pip install pygsheets

In [ ]:
#pip install prophet

In [ ]:
#pip install neuralprophet

In [ ]:
pip install xlsxwriter


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.7/152.7 kB 3.8 MB/s eta 0:00:00


In [ ]:
#from prophet import Prophet
#from prophet.plot import plot
#from prophet.plot import add_changepoints_to_plot
#from prophet.plot import plot_plotly, plot_components_plotly
#from prophet.diagnostics import cross_validation
##from prophet.diagnostics import performance_metrics
#from prophet.plot import plot_cross_validation_metric
#from neuralprophet import NeuralProphet
#import pyramid
#import pmdarima as pm

#from pmdarima.arima import auto_arima
#from pyramid.arima.stationarity import ADFTest
#import plotly.graph_objects as go

#import statsmodels.api as sm
#from statsmodels.tsa.seasonal import seasonal_decompose
#from statsmodels.tsa.stattools import adfuller
#from statsmodels.tsa.arima_model import ARIMA

#from sklearn.model_selection import TimeSeriesSplit
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, GradientBoostingRegressor
#from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import GridSearchCV, train_test_split, cross_validate
#from sklearn.linear_model import LinearRegression, RidgeCV,  LassoCV
#from sklearn import svm
#from sklearn.model_selection import cross_val_score

#import pytorch

#import keras
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM


import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import datetime
from datetime import date
import holidays
from datetime import timedelta
from pandas.tseries.offsets import MonthEnd
import warnings
import seaborn as sns
#import pygsheets 
import itertools
import functools
warnings.filterwarnings('ignore')
print( os.getcwd())

import google.auth
import google.cloud as gcp
from google_auth_oauthlib import flow
from google.cloud import storage
from google.cloud import bigquery
from oauth2client import file 

from pathlib import Path

#pd.set_option('max_columns', None)

/content


In [ ]:
def date_index(df):        
    '''pass dataframe and set datetime index'''
    if df.columns.str.contains('date').any() == True:
        df.set_index('date', inplace = True)
        
    if df.columns.str.contains('Date').any() == True:
        df.set_index('Date', inplace = True)
    
    if df.columns.str.contains('day_dt').any() == True:
        df.set_index('day_dt', inplace = True)

    if df.columns.str.contains('ds').any() == True:
        df.set_index('ds', inplace = True)
        
    #else:
     #   print('There does not appear to be a date like column')
    
    return df

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
project_id = 'rv-mt-data-prod'

das_df = pd.io.gbq.read_gbq("""
WITH wday AS(
SELECT accounting_dt AS day_dt,
EXTRACT(MONTH from accounting_dt) AS month_dt,
EXTRACT(QUARTER FROM accounting_dt) AS quarter_dt,
EXTRACT(YEAR from accounting_dt) AS year_dt,
company,


CASE WHEN LEFT(company,5) ='RV025' THEN 'US'
WHEN LEFT(company,5) ='RV028' THEN 'EMEA'
WHEN LEFT(company,5) ='RV026' THEN 'ANZ'
WHEN LEFT(company,5) ='RV027' THEN 'APAC'
WHEN LEFT(company,5) ='RV009' THEN 'US'
WHEN LEFT(company,5) ='RV013' THEN 'US'
WHEN LEFT(company,5) ='RV001' THEN 'Red Ventures'
WHEN LEFT(company,5) ='RV020' THEN 'Red Ventures'
WHEN LEFT(company,5) ='RV029' THEN 'Channel Community Network Corporation'
WHEN CONCAT(channel," _ ",revenue_category) IN("C501 Programmatic - Guaranteed _ Desktop","C501 Programmatic - Guaranteed _ Mobile Exchange","C501 Programmatic - Guaranteed _ Desktop Exchange","C501 Programmatic - Guaranteed _ Video","C501 Programmatic - Guaranteed _ Mobile","C501 Programmatic - Guaranteed") AND LEFT(company,5) ='RV026' THEN 'US'
WHEN LEFT(company,5) ='RV007' THEN 'Soda'
WHEN LEFT(company,5) ='RV022' THEN 'My Slumber Yard'
WHEN LEFT(company,5) ='RV009' THEN 'Cards (formerly Bankrate)'
WHEN LEFT(company,5) ='RV045' THEN 'Bankrate)'
ELSE LEFT(company,5) END AS region,

journal_number,
journal_name,
status AS ledger_status,
source_desc AS ledger_source,
ledger AS ledger_type,


operational_transaction,
line_memo,
CASE WHEN line_memo = 'Twitter Amplify May' THEN '81945'
WHEN line_memo LIKE ('%Twitter%') THEN '81760'
WHEN line_memo like('%reclass%') THEN 'Rev Reclass'
WHEN line_memo like('%Reclass%') THEN 'Rev Reclass'
WHEN line_memo like('%AU Enterprise%') THEN 'AU Enterprise'
WHEN line_memo like('%CCN: Google%') THEN 'CCN Google'
#WHEN line_memo like('CM|a||%') THEN 'No ID'
#WHEN line_memo like('%CM|%') THEN 'No ID'
WHEN line_memo like('%IBM IPC%') THEN 'IBM'
WHEN line_memo like('%IO#%') THEN SUBSTR(line_memo,5,5)
WHEN line_memo like('%NoCable:%') THEN 'No Cable'
ELSE LEFT(line_memo,5) END AS operative_order_id,
channel,
revenue_category,
CONCAT(channel," _ ",revenue_category) AS channel_revcat_cat,

CASE WHEN CONCAT(channel," _ ",revenue_category) IN (
"C010 Production Fees _ Other",
"C121 Video - Onsite _ Video",
"C265 Display _ Advertising -Digital",
#"C265 Display _ Advertising -Digital"
"C265 Display _ Desktop",
"C265 Display _ Mobile",
"C726 Email/Co-Reg _ Other",
"C957 Social Media _ Other",
"C957 Social Media _ Video"
) THEN 'Direct Ad Sales'

WHEN CONCAT(channel," _ ",revenue_category) IN(
  "C501 Programmatic - Guaranteed _ Desktop",
  #"C501 Programmatic - Guaranteed _ Mobile Exchange",
  #"C501 Programmatic - Guaranteed _ Desktop Exchange",
  "C501 Programmatic - Guaranteed _ Video",
  "C501 Programmatic - Guaranteed _ Mobile"
  #"C501 Programmatic - Guaranteed"
  ) THEN 'Programmatic Guaranteed'

WHEN CONCAT(channel," _ ",revenue_category) IN ("C002 Audio _ Desktop Exchange",
"C500 Programmatic _ Desktop Exchange",
"C500 Programmatic _ Desktop",
"C726 Email/Co-Reg _ Desktop Exchange",
"C500 Programmatic _ Mobile Exchange",
"C500 Programmatic _ Video Exchange",
"C500 Programmatic _ Video",
"C501 Programmatic - Guaranteed _ Mobile Exchange",
"C501 Programmatic - Guaranteed _ Desktop Exchange"
) THEN 'Open Exchange'

WHEN channel LIKE('%PMP%')# IN ("Programmatic PMP _ Desktop Exchange","Programmatic PMP _ #Mobile Exchange","Programmatic PMP _ Video Exchange") 
THEN 'PMP'

WHEN customer IN ('YouTube, LLC','Facebook Alpha Product','Google AdSense Service Agreement') THEN 'Other'
WHEN CONCAT(channel," _ ",revenue_category) IN ("C528 Paid Search") THEN 'Other'
WHEN business_unit IN ('B761 Giantbomb','B759 Gamefaqs','B774 Comic Vine','B760 Gamespot') AND CONCAT(channel," _ ",revenue_category) IN ('C000 Admin _ Subscriptions',
'C280 Other _ Merchandise',
'C280 Other _ Subscriptions',
'C120 Video – Offsite _ Subscriptions',
'C120 Video – Video',
'C120 Video – Video Exchange') THEN 'Other'
WHEN business_unit IN ('B762 TVGuide','B763 Metacritic','B765 TV.com','B766 Chowhound','B767 Guide.com','B771 Soda','B772 nocable','B773 CordCutters') AND CONCAT(channel," _ ",revenue_category) IN ('C000 Admin _ DAS','C120 Video – Offsite _ Video Exchange','C280 Other _ Licensing','C280 Other _ Other','C334 AllConnect.com _Advertising -Digital') THEN 'Other'

--Licensing Other
WHEN CONCAT(channel," _ ",revenue_category) IN ('C594 Partners _ Licensing',
'C594 Partners _ Activation / Installation', 'C280 Other _ Licensing','C280 Other _ Other') THEN 'Other'
WHEN business_unit LIKE('%CNET%') AND customer NOT LIKE ('%Faceb%') AND CONCAT(channel," _ ",revenue_category) IN ('C000 Admin _  Activation / Installation','C000 Admin _  Chargeback Expense (Contra Revenue)') THEN 'Other'
WHEN business_unit LIKE('%Download%') AND customer NOT LIKE ('%Faceb%') AND CONCAT(channel," _ ",revenue_category) IN ('C000 Admin _  Activation / Installation','C000 Admin _  Chargeback Expense (Contra Revenue)') THEN 'Other'

--transactional ecomm
WHEN CONCAT(channel," _ ",revenue_category) IN ('C594 Partners _ Leads',
'C594 Partners _ Leads - Exclusive','C594 Partners _ Other') THEN 'Commerce'


ELSE revenue_category END AS revenue_type_rollup,

business_unit,
CASE WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) = 'CMG' THEN 'CMG'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%TPG%')  THEN 'The Points Guy'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Vine%') THEN 'Comic Vine'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) = 'Giantbomb' THEN 'Giant Bomb'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) = 'TIME' THEN 'Next Advisor'

WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%faqs%') THEN 'Game FAQs'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%spot%') THEN 'Game Spot'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Technology%') THEN 'CNET Tech'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Home%') THEN 'CNET Home'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Road%') THEN 'CNET Roadshow'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Exp%') THEN 'CNET Expansion'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%B2B - ZD%') THEN 'ZD Net'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%B2B - Tech%') THEN 'Tech Republic'
WHEN business_unit = 'B767 Guide.com' THEN 'Guide.com'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%TVGuide%') THEN 'TV Guide'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Downl%') THEN 'Download'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Guide.com%') THEN 'Guide.com'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Chow%') THEN 'Chowhound'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Republic%') THEN 'Tech Republic'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Higher%') THEN 'EDU'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%ollege%') THEN 'EDU'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Education%') THEN 'EDU'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Learn%') THEN 'EDU'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Degree%') THEN 'EDU'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%school%') THEN 'EDU'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%grad%') THEN 'EDU'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Grad%') THEN 'EDU'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%General%') THEN 'EDU'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Soda%') THEN 'Soda'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%cable%') THEN 'No cable'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Cord%') THEN 'Cord Cutters'
WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Bank%') THEN 'Bank Rate'
ELSE RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) END AS brand,





CASE WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%CNET%') 
  AND LEFT(company,5) ='RV025' THEN 'CNET'
    WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%PriceBlink%') 
  AND LEFT(company,5) ='RV025' 
  AND CONCAT(channel," _ ",revenue_category) IN ('C594 Partners _ Leads')  THEN 'CNET'  
  
  WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Download%') 
  AND LEFT(company,5) ='RV025' THEN 'CNET'

     WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%CNET%') OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Down%')
  AND company NOT IN ("RV014 Healthline Holdings, Inc",
"RV015 Healthline UK",
"RV016 Healthline Iceland", 
"RV994 CNET Elimination", 
"RV999 Eliminations Company", 
"TR001 Tech Republic - US",
"TR002 Tech Republic - Australia",
"TR003 Tech Republic - Singapore",
"TR004 Tech Republic - UK",
"F001 Hudson Takeoff",
"RV025 CNET Media, Inc.") THEN 'CNET_ROW'

WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%ZD%') 
  AND LEFT(company,5) ='RV025' THEN 'B2B'

WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%ZD%') 
  AND LEFT(company,5) NOT IN ('F001','RV014','RV015', 'RV016','RV994', 'RV999', 'TR001','TR002','TR003','TR004','RV025','RV024', 'RV042') THEN 'B2B_ROW'

WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Game%') OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Giant%')OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Comic%') 
  AND LEFT(company,5) NOT IN ('F001', 'RV014','RV015', 'RV016','RV994', 'RV999', 'TR001','TR002','TR003','TR004','RV024', 'RV042') THEN 'Games'


WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%MySlumber%') OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Safety%')OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%TIME%') 
  AND LEFT(company,5) NOT IN ('F001','RV014', 'RV015', 'RV016','RV994', 'RV999', 'TR001','TR002','TR003','TR004','RV024', 'RV042') THEN 'Other'


WHEN RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%TV%') OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Meta%')OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Metro%') OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Metro%')
OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Chow%') OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Soda%') OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%noCable%') OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%Cord%')OR RIGHT(business_unit, CHARACTER_LENGTH(business_unit) - 5) LIKE('%nocable%')
OR business_unit = 'B767 Guide.com' 
AND LEFT(company,5) NOT IN ('F001','RV014', 'RV015', 'RV016','RV994', 'RV999', 'TR001','TR002','TR003','TR004','RV024', 'RV042') THEN 'Entertainment'

ELSE "" END AS brand_region,

CASE WHEN business_unit IN ('B752 CNET Movers','B755 CNET Technology','B756 CNET Roadshow','B757 CNET Expansion','B770 Download.com') THEN 'CNET_Tech'
WHEN business_unit IN ('B830 CNET Media') THEN 'CNET_Media'
WHEN business_unit IN ('B828 CNET Broadband','B829 CNET Energy','B095 CNET Money') THEN 'CNET_Others'
WHEN business_unit = 'B930 Product' THEN 'Product'
WHEN business_unit = 'B199 MySlumberYard.com' THEN 'MSY'
WHEN business_unit = 'B404 Safety.com' THEN 'Safety'
WHEN business_unit = 'B753 B2B - ZDNet' THEN 'ZD_Net'
WHEN business_unit = 'B758 B2B - TechRepublic' THEN 'TR'
WHEN business_unit IN ('B759 Gamefaqs','B760 Gamespot','B761 Giantbomb','B774 Comic Vine','B762 TVGuide','B763 Metacritic','B773 CordCutters','B764 Metrolyrics','B754 M&T Entertainment','B751 Gaming') THEN 'G&E(Fandom)'
WHEN business_unit IN ('B771 Soda','B772 nocable','B765 TV.com','B766 Chowhound','B767 Guide.com') THEN 'G&E'
WHEN business_unit IN ('B548 TIME','B531 Next Advisor') THEN 'Time'
WHEN business_unit IN ('B350 CMG','B811 Media & Commerce Allocation','B768 CCS Data Services','B769 CCS Channel Online','B824 CNET Central Allocation','B831 CNET Group - Corporate','B010 Corporate') THEN 'Corporate'
WHEN business_unit IN ('B701 Imagitas-USPS','B822 MyMove Allocation','B700 Imagitas Corporate','B711 Imagitas-My Move','B712 Mymove.com','B713 Movers Club','B714 MyCOA') THEN 'Movers'
WHEN business_unit IN ('B040 Save on Energy','B041 Save on Energy - Commercial','B047 Choose Energy','B050 Solar Marketplace','B049 TXU Energy') THEN 'Energy'
WHEN business_unit = 'B450 Allconnect.com' THEN 'Broadband'
ELSE "" END AS updated_bu_map,



ledger_account,

CASE WHEN ledger_account IN ('5103:Product Cost', '5112:Other Cost of Sales', '5116:Direct Marketing', '5254:Freight', '5350:Sales and Use Tax') THEN 'Marketing Expenses'

WHEN ledger_account IN ('7000:Payroll Wages','7005:Payroll Taxes','7010:Bonuses','7011:Other Compensation','7025:Temporary Help', '7050:Benefits') THEN 'Head-Count Expenses'

WHEN ledger_account IN ('6010:Agent Payroll Taxes','6015:Agent Benefits','6030:Non-Agent Payroll Taxes', '6050:Other Agent Comp','6400:Facility Sales Center','7060:Recruiting','7100:Facility General & Administrative','7130:Company Functions','7135:Travel','7140:Meals & Entertainment','7142:Catering General & Administrative','7150:Employee Motivation','7160:Supplies',
'7170:Postage and Courier','7204:Licensing and Fees','7230:Other General & Administrative','7250:Research','7300:Technology','7800:Bad Debt','7999:Allocated Costs other','8200:Professional Services','8205:Dues and Subscriptions','8230:Public Relations','8240:General Insurance','8290:Contributions','8500:Café Management') THEN 'Non Head-Count Expense'

WHEN ledger_account = '8995:Vertical Expense Allocation' THEN 'Vertical Allocation Expense'
WHEN ledger_account = '8999:Shared Expense Allocation' THEN 'Corporate Allocation Expense'

WHEN ledger_account IN (
'4000:Revenue',
'4005:Other Revenue',
'4132:Chargeback Expense (Contra Revenue)')
#"1102:Accrued Accounts Receivable")
THEN 'Revenue'
ELSE ledger_account END AS ledger_account_rollup,
cost_center,
customer,

SUM(translated_debit_minus_credit_amount) AS workday_revenue,
supplier_as_worktag,
spend_category_as_worktag,
spend_cat_id,





FROM `rv-mt-data-prod.workday_vw.workday_journal` 

# Yr changed to 2023 due to data size and time to pull
WHERE accounting_dt >= '2023-01-01'
#AND operational_transaction LIKE('%Customer Invoice%')

--turn on later
#AND ledger_account IN ('4000:Revenue','4005:Other Revenue','4132:Chargeback Expense (Contra Revenue)',"1102:Accrued Accounts Receivable")

AND company NOT IN("RV014 Healthline Holdings, Inc",
"RV015 Healthline UK",
"RV016 Healthline Iceland", 
"RV994 CNET Elimination", 
"RV999 Eliminations Company", 
"TR001 Tech Republic - US",
"TR002 Tech Republic - Australia",
"TR003 Tech Republic - Singapore",
"TR004 Tech Republic - UK",
"RV021 Forward Home Lending, LLC",
"RV039 Lonely Planet Publications Inc.")

#AND line_memo Like '%IO#%'
GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,28,29,30
#HAVING revenue_type_rollup IN ('Direct Ad Sales', 'Programmatic Guaranteed') 

--turn on later
#AND ledger_account_rollup = 'Revenue'
),

sf AS (
SELECT 
CASE WHEN account_category_nm IN ('B2B', 'Consumer Electronics') THEN 'Tech'
WHEN account_category_nm LIKE('%Incubator%') THEN 'Incubator'
WHEN account_category_nm LIKE('%Misc%') THEN 'Incubator'
WHEN account_category_nm LIKE('%Social%') THEN 'Incubator'
WHEN account_category_nm LIKE('%Beer%') THEN 'Incubator'
WHEN account_category_nm LIKE('%Spirits%') THEN 'Incubator'
WHEN account_category_nm LIKE('%CPG%') THEN 'Incubator'
WHEN account_category_nm LIKE('%QSR%') THEN 'Incubator'
WHEN account_category_nm LIKE('%Insurance%') THEN 'Incubator'
WHEN account_category_nm LIKE('%Travel%') THEN 'Incubator'
ELSE account_category_nm END AS category,
operative_nbr,
opportunity_nm
#EXTRACT(MONTH from schedule_month) AS month_nbr,
#round(SUM(opportunity_revenue),2) AS sfdc_revenue
FROM `rv-mt-data-prod.dw_vw.sfdc_opportunity_pipeline` 
WHERE schedule_month >='2023-01-01'
AND probability_pct >= 90

AND product_nm NOT IN('Programmatic - GameSpot Video','Asia: Direct: Basic Lead',
'Asia: Direct: Profiled Lead (1 to 4 questions)',
'AU: Direct: Basic Lead',
'AU: Direct: Direct Bypass',
'AU: Direct: Email Revenue',
'AU: Direct: Profiled Lead (1 to 4 questions)',
'AU: Direct: Profiled Lead (5 to 7 questions)',
'B2B Lead Gen',
'UK: Direct: Basic Lead',
'UK: Direct: Highly Qualified Lead',
'UK: Direct: Profiled Lead (1 to 4 questions)')

AND opportunity_owner NOT IN('chinedu okadigbo', 'Craig Abley','Harry Michael','Hetvi Desai','Jackie McCarthy','Jahuie {Ja-Who-E} McCrary', 'Janet Tomokuni','Joe Mills','Kay Hope','Kevin Lichtman','Michael Macias','Nina Abuel','Nixon Soh','Nonsie Mtimkulu',
'Patrick Steele','Preye Tobolayefa','Roberto Sanha','Ross Arthurs','Ruby Anwar','Tanusha Sharma','Vivean Cheah')

GROUP BY 1,2,3
ORDER BY 1
)

SELECT 
wday.day_dt,
wday.month_dt,
wday.quarter_dt,
wday.year_dt,
wday.company,
wday.region,

wday.journal_number,
wday.journal_name,
wday.ledger_status,
wday.ledger_source,
wday.ledger_type,
wday.operational_transaction,
wday.line_memo,
#CASE WHEN line_memo LIKE ('%IO 82471%') THEN '82471'
#ELSE wday.operative_order_id END AS 
operative_order_id,
wday.channel_revcat_cat,

CASE #WHEN wday.brand_region = 'CNET' AND wday.channel_revcat_cat = 'C280 Other _ Licensing' THEN 'Direct Ad Sales'
WHEN wday.brand_region = 'CNET_ROW' AND wday.month_dt = 1 AND wday.year_dt = 2022 AND wday.channel_revcat_cat IN(TRIM('C280 Other _ Licensing')) THEN 'Other'
WHEN wday.brand_region = 'CNET_ROW' AND wday.channel_revcat_cat = 'C280 Other _ Licensing' THEN 'Direct Ad Sales'
WHEN wday.brand_region = 'CNET_ROW' AND wday.channel_revcat_cat = 'C000 Admin _ Other' THEN 'Other'
WHEN wday.brand_region = 'Other' AND wday.channel_revcat_cat = 'C265 Display _ Other' THEN 'Direct Ad Sales'
WHEN wday.brand_region = 'Other' AND wday.channel_revcat_cat ='C125 TIME Partnership _ Activation / Installation' THEN 'Commerce'
WHEN wday.brand_region = 'Entertainment' AND wday.channel_revcat_cat = 'C265 Display _ Advertising -Digital' THEN 'Direct Ad Sales'


WHEN wday.brand_region = 'Other' AND wday.channel_revcat_cat = 'C198 Natural Search _ Leads' THEN 'Commerce'
WHEN wday.brand_region = 'Other' AND wday.channel_revcat_cat ='C400 Affiliate _ Activation / Installation' THEN 'Commerce'
WHEN wday.brand_region = 'Other' AND wday.channel_revcat_cat = 'C407 MyFinance - RVMN _ Activation / Installation'  THEN 'Commerce'

WHEN wday.brand_region = 'Other' AND wday.channel_revcat_cat = 'C408 MyFinance - O&O - Bankrate _ Clicks' THEN 'Commerce'
#WHEN wday.brand_region = 'CNET' AND wday.month_dt = 2 AND wday.channel_revcat_cat IN('C501 Programmatic - Guaranteed _ Desktop Exchange','C501 Programmatic - Guaranteed _ Mobile Exchange') THEN 'Direct Ad Sales'
WHEN wday.brand_region = 'CNET' AND wday.month_dt = 6 AND wday.channel_revcat_cat IN('C280 Other _ Other') THEN 'Direct Ad Sales'
WHEN wday.brand_region = 'Other' AND wday.month_dt = 6 AND wday.channel_revcat_cat IN('C957 Social Media _ Desktop') THEN 'Direct Ad Sales'
WHEN wday.brand_region = 'CNET' AND wday.month_dt = 2 AND wday.channel_revcat_cat IN('C501 Programmatic - Guaranteed _ Desktop Exchange','C501 Programmatic - Guaranteed _ Mobile Exchange') THEN 'Programmatic Guaranteed'
WHEN wday.brand_region = 'B2B' AND wday.month_dt = 2 AND wday.channel_revcat_cat IN('C501 Programmatic - Guaranteed _ Desktop Exchange','C501 Programmatic - Guaranteed _ Mobile Exchange') THEN 'Programmatic Guaranteed'
WHEN wday.brand_region = 'Games' AND wday.month_dt = 2 AND wday.channel_revcat_cat IN('C501 Programmatic - Guaranteed _ Desktop Exchange','C501 Programmatic - Guaranteed _ Mobile Exchange') THEN 'Programmatic Guaranteed'
WHEN wday.brand_region = 'Entertainment' AND wday.month_dt = 2 AND wday.channel_revcat_cat IN('C501 Programmatic - Guaranteed _ Desktop Exchange','C501 Programmatic - Guaranteed _ Mobile Exchange') THEN 'Programmatic Guaranteed'
WHEN wday.brand_region = 'CNET_ROW' AND wday.month_dt = 3 AND wday.channel_revcat_cat IN('C280 Other _ Desktop') THEN 'Direct Ad Sales'
WHEN wday.brand_region = 'B2B' AND wday.month_dt = 3 AND wday.channel_revcat_cat IN('C726 Email/Co-Reg _ Leads') THEN 'Direct Ad Sales'
#WHEN wday.brand_region = 'CNET_ROW' AND wday.month_dt = 1 AND wday.channel_revcat_cat IN(TRIM('C280 Other _ Licensing')) THEN 'Other'
WHEN wday.brand_region = 'B2B_ROW' AND wday.month_dt = 1 AND wday.channel_revcat_cat IN('C280 Other _ Other') THEN 'Direct Ad Sales'
WHEN wday.brand_region = 'CNET' AND wday.month_dt = 7 AND wday.channel_revcat_cat IN('C280 Other _ Other') THEN 'Direct Ad Sales'
WHEN wday.year_dt >= 2022 AND wday.month_dt >= 8 AND wday.channel_revcat_cat IN('C120 Video - Offsite _ Video') THEN 'Direct Ad Sales'

WHEN wday.brand_region = 'Other' AND wday.year_dt >= 2022 AND wday.month_dt >= 8 AND wday.channel_revcat_cat IN('C500 Programmatic _ Mobile') THEN 'Open Exchange'
WHEN wday.brand_region = 'B2B' AND wday.year_dt >= 2022 AND wday.month_dt >= 8 AND wday.channel_revcat_cat IN('C594 Partners _ Clicks') THEN 'Commerce'
WHEN wday.brand_region = 'Entertainment' AND wday.year_dt >= 2022 AND wday.month_dt >= 8 AND wday.channel_revcat_cat IN('C334 AllConnect.com _ Advertising -Digital') THEN 'Commerce'
WHEN wday.brand_region = 'Other' AND wday.year_dt >= 2022 AND wday.month_dt >= 7 AND wday.channel_revcat_cat IN('C594 Partners _ Advertising -Digital','C594 Partners _ Activation / Installation','C594 Partners _ Chargeback Expense (Contra Revenue)') THEN 'Commerce'
WHEN wday.brand_region = 'Games' AND wday.year_dt >= 2022 AND wday.month_dt = 7 AND wday.channel_revcat_cat IN('C002 Audio _ Desktop Exchange') THEN 'Other'
WHEN wday.brand_region = 'Entertainment' AND wday.year_dt >= 2022 AND wday.month_dt = 7 AND wday.channel_revcat_cat IN('C500 Programmatic _ Other') THEN 'Open Exchange'
WHEN wday.brand_region = 'Other' AND wday.year_dt >= 2022 AND wday.month_dt >= 7 AND wday.channel_revcat_cat IN('C500 Programmatic _ Mobile') THEN 'Open Exchange'

WHEN wday.brand_region = 'Games' AND wday.year_dt >= 2022 AND wday.month_dt = 8 AND wday.channel_revcat_cat IN('C002 Audio _ Desktop Exchange') THEN 'Other'
WHEN wday.brand_region = 'Games' AND wday.year_dt >= 2022 AND wday.month_dt >= 9 AND wday.channel_revcat_cat IN ('C002 Audio _ Other') THEN 'Open Exchange'
WHEN wday.brand_region = 'Games' AND wday.year_dt >= 2022 AND wday.month_dt >= 9 AND wday.channel_revcat_cat IN ('C002 Audio _ Desktop Exchange') THEN 'Open Exchange'
WHEN wday.brand_region = 'Entertainment' AND wday.year_dt >= 2022 AND wday.month_dt >= 9 AND wday.channel_revcat_cat IN ('C500 Programmatic _ Other') THEN 'Open Exchange'
WHEN wday.brand_region = 'CNET' AND wday.year_dt >= 2022 AND wday.month_dt >= 10 AND wday.channel_revcat_cat IN('C280 Other _ Other') THEN 'Direct Ad Sales'

WHEN wday.brand_region = 'Other' AND wday.year_dt >= 2022 AND wday.month_dt >= 11 AND wday.channel_revcat_cat IN('C957 Social Media _ Desktop') THEN 'Direct Ad Sales'

WHEN wday.brand_region = 'CNET' AND wday.year_dt >= 2022 AND wday.month_dt >= 12 AND wday.channel_revcat_cat IN('C000 Admin _ Desktop') THEN 'Direct Ad Sales'


ELSE wday.revenue_type_rollup END AS revenue_type_rollup,
wday.business_unit,
wday.brand,

CASE WHEN wday.brand = 'PriceBlink' AND wday.ledger_account_rollup 
IN ('Non Head-Count Expense', "Head-Count Expenses") THEN 'CNET'
ELSE wday.brand_region END AS brand_region,
wday.ledger_account,
wday.ledger_account_rollup,
wday.customer,
wday.cost_center,

CASE WHEN wday.operative_order_id = '80709' THEN 'Tech'
WHEN wday.operative_order_id = '80137' THEN 'Tech'
WHEN wday.operative_order_id = '81000' THEN 'Finance'
WHEN wday.operative_order_id = '80223' THEN 'Consumer Electronics'
WHEN wday.operative_order_id = '05.20' THEN 'Finance'
WHEN wday.operative_order_id = 'VCBS' THEN 'M&E'
WHEN wday.operative_order_id = '82080' THEN 'Finance'
WHEN wday.operative_order_id = '80727' THEN 'M&E'
WHEN wday.operative_order_id = '80403' THEN 'M&E'
WHEN wday.operative_order_id = '81692' THEN 'Retail'
WHEN wday.operative_order_id = '82064' THEN 'M&E'
WHEN wday.operative_order_id = 'IBM' THEN 'Tech'
WHEN wday.line_memo = '07.2022 Cap One intercompany order' AND month_dt = 7 THEN 'Finance'
WHEN wday.operative_order_id = '81137' AND month_dt = 7 THEN 'Gaming'
WHEN wday.operative_order_id = '81136' AND month_dt = 7 THEN 'Gaming'
WHEN wday.line_memo = 'VCBS H1 FORFEIT Marketing credit' AND month_dt = 6 THEN 'M&E'
WHEN wday.operative_order_id = TRIM('Rev Reclass') AND month_dt = 6 THEN 'Finance'
WHEN wday.operative_order_id = TRIM('81481') AND month_dt = 6 THEN 'Retail'
WHEN wday.operative_order_id = TRIM('81547') AND month_dt = 6 THEN 'Retail'
WHEN wday.operative_order_id = TRIM('81574') AND month_dt = 6 THEN 'Retail'
WHEN wday.operative_order_id = TRIM('81426') THEN 'Retail'
WHEN wday.line_memo = '08.2022 Cap One intercompany order'  THEN 'Finance'
WHEN wday.operative_order_id = TRIM('81662') THEN 'Retail'
WHEN wday.operative_order_id = TRIM('Huawe') THEN 'Telco'
WHEN wday.line_memo LIKE ('%HUAWEI%') THEN 'Telco'
WHEN wday.operative_order_id = TRIM('80156') THEN 'Incubator'
WHEN wday.operative_order_id = TRIM('Comic') THEN 'Gaming'
WHEN wday.operative_order_id = TRIM('80333') THEN 'Telco'
WHEN wday.operative_order_id = TRIM('81327') THEN 'Incubator'
WHEN wday.operative_order_id = TRIM('82163') THEN 'Telco'
WHEN wday.line_memo = '09.2022 Cap One intercompany order'  THEN 'Finance'
WHEN wday.line_memo = 'VCBS Q3 FORFEIT Marketing credit'  THEN 'M&E'
WHEN wday.line_memo = '10.2022 Cap One intercompany order' THEN 'Finance'
WHEN wday.operative_order_id = TRIM('OPTUS') THEN 'Telco'

ELSE sf.category END AS category,
CASE WHEN wday.operative_order_id = '80709' THEN '47712_Google_AdX_RevShare_PG'	
ELSE sf.opportunity_nm END AS opportunity_nm,

SUM(wday.workday_revenue) AS workday_rev_actuals,
wday.supplier_as_worktag,
wday.spend_category_as_worktag,
wday.spend_cat_id,

CASE WHEN wday.brand IN( 'Entertainment','Games','Metacritic','TV.com') AND wday.year_dt >= 2022 AND wday.month_dt >= 10 THEN 'Fandom'
ELSE 'RV' END AS exclusion_filter,
updated_bu_map,

#CNET MEDIA
CASE WHEN wday.business_unit = 'B830 CNET Media' and wday.channel_revcat_cat IN ( 
  'C010 Production Fees _ Other',
'C121 Video - Onsite _ Video',
'C265 Display _ Desktop',
'C265 Display _ Mobile',
#'C280 Other _ Licensing',
'C501 Programmatic - Guaranteed _ Desktop',
'C501 Programmatic - Guaranteed _ Mobile',
'C501 Programmatic - Guaranteed _ Video',
'C501 Programmatic - Guaranteed _ Desktop Exchange',
'C501 Programmatic - Guaranteed _ Mobile Exchange',
'C501 Programmatic - Guaranteed _ Video Exchange',
'C265 Display _ Other',
'C265 Display _ Non-Direct')  THEN 'DAS_Onsite_Rev'

WHEN  wday.business_unit = 'B830 CNET Media' and wday.channel_revcat_cat IN ('C265 Display _ Advertising -Digital') and customer LIKE('%Viacom%')THEN 'DAS_Onsite_Rev'


WHEN wday.business_unit = 'B830 CNET Media' and wday.channel_revcat_cat IN ('C957 Social Media _ Other') THEN 'DAS_Offsite_Rev(Social)'

WHEN wday.business_unit = 'B830 CNET Media' and wday.channel_revcat_cat IN (
'C002 Audio _ Desktop Exchange',
'C500 Programmatic _ Desktop',
'C500 Programmatic _ Desktop Exchange',
'C500 Programmatic _ Mobile Exchange',
'C500 Programmatic _ Video Exchange',
'C500 Programmatic _ Video',
'C500 Programmatic _ Other',
'C528 Paid Search _ Clicks',
'C528 Paid Search _ Other',
'C726 Email/Co-Reg _ Desktop Exchange'
) THEN 'DAS_Onsite_Backfill_Rev(OE)'

WHEN wday.business_unit = 'B830 CNET Media' and wday.channel_revcat_cat IN ('Programmatic PMP _ Desktop Exchange',
'Programmatic PMP _ Mobile Exchange','Programmatic PMP _ Video Exchange') THEN 'DAS_Onsite_Backfill_Rev(PMP)'



WHEN wday.business_unit = 'B830 CNET Media' and wday.channel_revcat_cat IN ('C120 Video - Offsite _ Video Exchange','C120 Video - Offsite _ Licensing') THEN 'DAS_Offsite_Backfill_Rev(YT)'
WHEN wday.business_unit = 'B830 CNET Media' and wday.channel_revcat_cat IN ('C594 Partners _ Licensing') THEN 'Licensing_Rev_Intl'
WHEN wday.business_unit = 'B830 CNET Media' and wday.channel_revcat_cat IN ('C280 Other _ Licensing') THEN 'Licensing_Rev_Brand'
WHEN wday.business_unit = 'B830 CNET Media' and wday.channel_revcat_cat IN ('C726 Email/Co-Reg _ Desktop Exchange','C726 Email/Co-Reg _ Other') THEN 'Email_Media_Rev'

#CNET Tech
WHEN wday.business_unit IN ('B752 CNET Movers','B755 CNET Technology','B756 CNET Roadshow','B757 CNET Expansion','B770 Download.com') AND wday.channel_revcat_cat ='C594 Partners _ Leads' THEN 'Transactional'

#CNET Others
WHEN wday.business_unit IN ('B828 CNET Broadband','B829 CNET Energy','B095 CNET Money') AND wday.channel_revcat_cat  IN('C470 CMG _ Clicks','C470 CMG _ Leads','C594 Partners _ Activation / Installation',
'C594 Partners _ Advertising -Digital',
'C594 Partners _ Chargeback Expense (Contra Revenue)', 
'C594 Partners _ Cart Compensation') THEN 'Transactional'

#ZD Net Original:
#WHEN wday.business_unit IN ('B753 B2B - ZDNet') AND wday.channel_revcat_cat  IN('C121 Video - Onsite _ Video','C265 Display _ Desktop','C265 Display _ Mobile','C726 Email/Co-Reg _ Other','C957 Social Media _ Other') THEN 'US_Direct_Ad_Sales(Discover)'
#WHEN wday.business_unit IN ('B753 B2B - ZDNet') AND wday.channel_revcat_cat  IN('C002 Audio _ Desktop Exchange','C500 Programmatic _ Desktop','C500 Programmatic _ Desktop Exchange','C500 Programmatic _ Mobile Exchange','C500 Programmatic _ Video Exchange',
#'C726 Email/Co-Reg _ Desktop Exchange', 'Programmatic PMP _ Desktop Exchange','Programmatic PMP _ Mobile Exchange','Programmatic PMP _ Video Exchange') THEN 'US_Programmatic(Discover)'
#WHEN wday.business_unit IN ('B753 B2B - ZDNet') AND wday.channel_revcat_cat  IN('C280 Other _ Licensing','C594 Partners _ Clicks','C594 Partners _ Desktop Exchange','C594 Partners _ Licensing') THEN 'US_Licensing(Discover)'
#WHEN wday.business_unit IN ('B753 B2B - ZDNet') AND wday.channel_revcat_cat  IN('C594 Partners _ Leads','C594 Partners _ Leads - Exclusive') THEN 'US_Commerce(Discover)'


WHEN wday.business_unit = 'B753 B2B - ZDNet' and wday.channel_revcat_cat IN ( 
  'C010 Production Fees _ Other',
'C121 Video - Onsite _ Video',
'C265 Display _ Desktop',
'C265 Display _ Mobile',
#'C280 Other _ Licensing',
'C501 Programmatic - Guaranteed _ Desktop',
'C501 Programmatic - Guaranteed _ Mobile',
'C501 Programmatic - Guaranteed _ Video',
'C501 Programmatic - Guaranteed _ Desktop Exchange',
'C501 Programmatic - Guaranteed _ Mobile Exchange',
'C501 Programmatic - Guaranteed _ Video Exchange',
'C265 Display _ Other',
'C265 Display _ Non-Direct')  THEN 'DAS_Onsite_Rev'

WHEN wday.business_unit = 'B753 B2B - ZDNet' and wday.channel_revcat_cat IN ('C957 Social Media _ Other') THEN 'DAS_Offsite_Rev(Social)'

WHEN wday.business_unit = 'B753 B2B - ZDNet' and wday.channel_revcat_cat IN (
'C002 Audio _ Desktop Exchange',
'C500 Programmatic _ Desktop',
'C500 Programmatic _ Desktop Exchange',
'C500 Programmatic _ Mobile Exchange',
'C500 Programmatic _ Video Exchange',
'C500 Programmatic _ Video',
'C500 Programmatic _ Other',
'C528 Paid Search _ Clicks',
'C528 Paid Search _ Other',
'C726 Email/Co-Reg _ Desktop Exchange'
) THEN 'DAS_Onsite_Backfill_Rev(OE)'

WHEN wday.business_unit = 'B753 B2B - ZDNet' and wday.channel_revcat_cat IN ('Programmatic PMP _ Desktop Exchange',
'Programmatic PMP _ Mobile Exchange','Programmatic PMP _ Video Exchange') THEN 'DAS_Onsite_Backfill_Rev(PMP)'



WHEN wday.business_unit = 'B753 B2B - ZDNet' and wday.channel_revcat_cat IN ('C120 Video - Offsite _ Video Exchange','C120 Video - Offsite _ Licensing') THEN 'DAS_Offsite_Backfill_Rev(YT)'
WHEN wday.business_unit = 'B753 B2B - ZDNet' and wday.channel_revcat_cat IN ('C594 Partners _ Licensing') THEN 'Licensing_Rev_Intl'
WHEN wday.business_unit = 'B753 B2B - ZDNet' and wday.channel_revcat_cat IN ('C280 Other _ Licensing') THEN 'Licensing_Rev_Brand'
WHEN wday.business_unit = 'B753 B2B - ZDNet' and wday.channel_revcat_cat IN ('C726 Email/Co-Reg _ Desktop Exchange','C726 Email/Co-Reg _ Other') THEN 'Email_Media_Rev'









#Product
WHEN wday.business_unit IN ('B930 Product') AND wday.channel_revcat_cat IN('C594 Partners _ Leads') THEN 'Shopping'
WHEN wday.business_unit IN ('B930 Product') AND wday.channel_revcat_cat IN('C594 Partners _ Leads') THEN 'Coupons'

#MSY & Time
WHEN wday.business_unit IN ('B199 MySlumberYard.com','B404 Safety.com','B548 TIME') AND wday.channel_revcat_cat  IN('C500 Programmatic _ Desktop','C500 Programmatic _ Mobile') THEN 'Programmatic'
WHEN wday.business_unit IN ('B199 MySlumberYard.com','B404 Safety.com','B548 TIME') AND wday.channel_revcat_cat  IN('C410 MyFinance - Publisher Network _ Clicks') THEN 'Other'
WHEN wday.business_unit IN ('B199 MySlumberYard.com','B404 Safety.com','B548 TIME') AND wday.channel_revcat_cat  IN('C125 TIME Partnership _ Activation/Installation', 'C198 Natural Search _ Leads','C265 Display _ Desktop','C400 Affiliate _ Activation/Installation',
'C407 MyFinance - RVMN _ Activation/Installation','C408 MyFinance - O&O - Bankrate _ Clicks','C594 Partners _ Activation/Installation','C594 Partners _ Advertising - Digital','C594 Partners _ Chargeback Expense (Contra Revenue)') THEN 'Transactional'

ELSE updated_bu_map END AS updated_rev_type_rollup,

#New ledger rollups
CASE WHEN wday.ledger_account IN (
#'5116:Direct Marketing',
#'5350:Sales and Use Tax',
#'6010:Agent Payroll Taxes',
#'6015:Agent Benefits',
#'6400:Facility Sales Center',
#'7000:Payroll Wages',
#'7005:Payroll Taxes',
#'7010:Bonuses',
#'7011:Other Compensation',
#'7025:Temporary Help', 
#'7050:Benefits',


'5112:Other Cost of Sales',
'7060:Recruiting',
'7100:Facility General & Administrative',
'7130:Company Functions',
'7135:Travel',
'7140:Meals & Entertainment',
'7142:Catering General & Administrative',
'7150:Employee Motivation',
'7160:Supplies',
'7170:Postage and Courier',
'7204:Licensing and Fees',
'7230:Other General & Administrative',
'7250:Research',
'7300:Technology',
'7800:Bad Debt',
'7999:Allocated Costs other',
'8200:Professional Services',
'8205:Dues and Subscriptions',
'8230:Public Relations',
'8240:General Insurance',
'8290:Contributions',
'8500:Café Management'
)  #AND wday.business_unit NOT IN('B010 Corporate')
THEN 'm&c_hc_expenses'
ELSE ledger_account END AS expense_flag,

wday.business_unit AS business_unit_original


FROM wday
LEFT JOIN sf
ON wday.operative_order_id = sf.operative_nbr


GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,27,28,29,30,31,32,33,34
#HAVING wday.ledger_account_rollup = 'Revenue' OR wday.ledger_account_rollup ='1102
#:Accrued Accounts Receivable'
ORDER BY year_dt DESC


""", project_id=project_id)

In [ ]:
das_df['opportunity_nm']= das_df['opportunity_nm'].fillna("blank")

In [ ]:
das_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115937 entries, 0 to 115936
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   day_dt                     115937 non-null  dbdate 
 1   month_dt                   115937 non-null  Int64  
 2   quarter_dt                 115937 non-null  Int64  
 3   year_dt                    115937 non-null  Int64  
 4   company                    115937 non-null  object 
 5   region                     115937 non-null  object 
 6   journal_number             30003 non-null   object 
 7   journal_name               112387 non-null  object 
 8   ledger_status              115937 non-null  object 
 9   ledger_source              115937 non-null  object 
 10  ledger_type                115937 non-null  object 
 11  operational_transaction    67688 non-null   object 
 12  line_memo                  40536 non-null   object 
 13  operative_order_id         40

In [ ]:
das_df.updated_rev_type_rollup.unique()

array(['CNET_Media', 'ZD_Net', 'MSY', '', 'Corporate', 'G&E', 'Product',
       'CNET_Tech', 'Movers', 'Broadband', 'CNET_Others', 'G&E(Fandom)',
       'TR', 'DAS_Onsite_Rev', 'Licensing_Rev_Intl',
       'DAS_Onsite_Backfill_Rev(OE)', 'Transactional',
       'DAS_Offsite_Rev(Social)', 'Email_Media_Rev',
       'DAS_Onsite_Backfill_Rev(PMP)', 'Shopping', 'Licensing_Rev_Brand',
       'DAS_Offsite_Backfill_Rev(YT)', 'Time', 'Programmatic', 'Safety',
       'Other', 'Energy'], dtype=object)

In [ ]:

def mth_filter(df,month_nm):
    '''dynmamic date filter.  Will return data for month selection'''
    #target_mth = input().astype(int)
    print ('dataframe will be filtered and populate month:',month_nm)
    df = df[(df['month_dt'].eq(month_nm))]
   
    return df


def mth_yr_filter(df,month_nm,year_nm):
    '''dynmamic date filter.  Will return data for month and year selection'''
    #target_mth = input().astype(int)
    print ('dataframe will be filtered and populate month:',month_nm,'and calendar year:',year_nm)
    df = df[(df['month_dt'].eq(month_nm))&(das_df['year_dt'].eq(year_nm))]
   
    return df




    #das_df[(das_df['month_dt'].eq(1))& (das_df['year_dt'].eq(2023))]

In [ ]:
das_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115937 entries, 0 to 115936
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   day_dt                     115937 non-null  dbdate 
 1   month_dt                   115937 non-null  Int64  
 2   quarter_dt                 115937 non-null  Int64  
 3   year_dt                    115937 non-null  Int64  
 4   company                    115937 non-null  object 
 5   region                     115937 non-null  object 
 6   journal_number             30003 non-null   object 
 7   journal_name               112387 non-null  object 
 8   ledger_status              115937 non-null  object 
 9   ledger_source              115937 non-null  object 
 10  ledger_type                115937 non-null  object 
 11  operational_transaction    67688 non-null   object 
 12  line_memo                  40536 non-null   object 
 13  operative_order_id         40

# Month Filter

In [ ]:
#das_df.loc[das_df['business_unit'].eq('B811 Media & Commerce Allocation'), 'business_unit'] = 'B811_B824_CNET_Corp'
#das_df.loc[das_df['business_unit'].eq('B824 CNET Central Allocation'), 'business_unit'] = 'B811_B824_CNET_Corp'


#moving BU's to Group Corp
das_df.loc[das_df['business_unit'].isin(['B350 CMG','B811 Media & Commerce Allocation','B824 CNET Central Allocation']), 'business_unit'] = 'CNET_group_corp'

#Companies for B010 only RV025,RV026,RV028,RV030
das_df.loc[(das_df['business_unit'].isin(['B010 Corporate']))& (das_df['company'].isin(['RV025 CNET Media, Inc','RV026 CNET Australia','RV028 CNET UK','RV030 Crimson Interactive GmbH'])), 'business_unit'] = 'CNET_group_corp'



das_df.loc[das_df['business_unit'].isin(['B770 Download.com','B752 CNET Movers','B756 CNET Roadshow','B757 CNET Expansion']), 'business_unit'] = 'B755_CNET_Technology'


#fill in all na and nan with blank for object and 0 for all else.  Helps with groupby and prevents omitting blank rows.  
#das_df = das_df.apply(lambda x : x.fillna('(blank)') if x.dtype == 'object' else x.fillna(0))
das_df.loc[das_df['supplier_as_worktag'].isna(), 'supplier_as_worktag'] = '(blank)'

das_df = mth_yr_filter(das_df,4,2023)
#copy
das_df2 = das_df

dataframe will be filtered and populate month: 4 and calendar year: 2023


In [ ]:
das_df['ledger_account_rollup'] = das_df['ledger_account_rollup'].str.replace(' ', '_')
das_df['ledger_account_rollup'] = das_df['ledger_account_rollup'].str.replace('-', '_')

das_df['business_unit'] = das_df['business_unit'].str.replace(' ', '_')
das_df['business_unit'] = das_df['business_unit'].str.replace('-', '_')
das_df['business_unit'] = das_df['business_unit'].str.replace('.', '_')
das_df['business_unit'] = das_df['business_unit'].str.replace('&', '_')


verticals = das_df.brand_region.unique()
verticals2 = das_df.updated_bu_map.unique()
ledger_rollups = das_df.ledger_account_rollup.unique()
ledger_rollups

bus_units = das_df.business_unit.unique()
bus_units

array(['CNET_group_corp', 'B755_CNET_Technology', 'B760_Gamespot',
       'B758_B2B___TechRepublic', 'B765_TV_com', 'B763_Metacritic',
       'B764_Metrolyrics', 'B774_Comic_Vine', 'B761_Giantbomb',
       'B759_Gamefaqs', 'B753_B2B___ZDNet', 'B766_Chowhound',
       'B830_CNET_Media', 'B762_TVGuide', 'B095_CNET_Money',
       'B450_Allconnect_com', 'B701_MyMove', 'B712_Mymove_com_(inactive)',
       'B700_MoverSource', 'B822_MyMove_Allocation',
       'B199_MySlumberYard_com', 'B930_Product', 'B754_M_T_Entertainment',
       'B827_RV_Home_Allocation_–_SAGA', 'B821_Home_Allocation',
       'B829_CNET_Energy', 'B828_CNET_Broadband', 'B548_TIME',
       'B010_Corporate', 'B215_Wild_Blue', 'B531_Next_Advisor',
       'B083_Red_Connect', 'B751_Gaming',
       'B711_Imagitas_My_Move_(inactive)', 'B404_Safety_com', 'B771_Soda',
       'B772_nocable', 'B831_CNET_Group___Corporate',
       'B173_Ingersoll_Rand', 'B096_XO', 'B768_CCS_Data_Services'],
      dtype=object)

In [ ]:
#das_df[(das_df['operative_order_id'].eq('82423')) & (das_df['ledger_account_rollup'].eq('Revenue'))]#.workday_rev_actuals.sum()#.info()

In [ ]:
das_working_rollups = ['Head_Count_Expenses','Marketing_Expenses', 'Non_Head_Count_Expense',
       'Vertical_Allocation_Expense', 'Corporate_Allocation_Expense','Revenue']
das_working_rollups

das_rev_rollups = ['Revenue']
das_exp_rollups = ['Head_Count_Expenses','Marketing_Expenses', 'Non_Head_Count_Expense',
       'Vertical_Allocation_Expense', 'Corporate_Allocation_Expense']

In [ ]:
def df_ledger_split(brand_region_name,ledger_account_rollups):
    '''creates a seperate df for each brand_region '''
    df = das_df[(das_df['brand_region'] == brand_region_name) & (das_df['ledger_account_rollup']== ledger_account_rollups)]
    df['ledger_account_rollup'] = df['ledger_account_rollup'].str.replace(' ', '_')
    
    return df  
    
for brand_region_name in verticals:
  for ledger_account_rollups in ledger_rollups:
    '''to call a vertical...CNET_ROW'''
    globals()[str(brand_region_name)+'_'+str(ledger_account_rollups)] = df_ledger_split(brand_region_name, ledger_account_rollups)




def df_split(brand_region_name):
    '''creates a seperate df for each brand_region '''
    df = das_df[(das_df['brand_region'] == brand_region_name)]
    
    return df  


for brand_region_name in verticals:
    '''to call a vertical...CNET_ROW'''
    globals()[str(brand_region_name)] = df_split(brand_region_name)

"""---------------------------------------------------------------------------------------------------------"""



def df_updated_ledger_split(updated_name,ledger_account_rollups):
    '''creates a seperate df for each updated brand '''
    df = das_df[(das_df['updated_bu_map'] == updated_name) & (das_df['ledger_account_rollup']== ledger_account_rollups)]
    df['ledger_account_rollup'] = df['ledger_account_rollup'].str.replace(' ', '_')
    
    return df  
    
for updated_name in verticals2:
  for ledger_account_rollups in ledger_rollups:
    '''to call a vertical...CNET_Tech'''
    globals()[str(updated_name)+'_'+str(ledger_account_rollups)] = df_updated_ledger_split(updated_name, ledger_account_rollups)




def updated_df_split(updated_name):
    '''creates a seperate df for each updated business unit '''
    df = das_df[(das_df['updated_bu_map'] == updated_name)]
    
    return df 





for updated_name in verticals2:
    '''to call a vertical...CNET_ROW'''
    globals()[str(updated_name)] = updated_df_split(updated_name)    


    """------------------------------------------------------------------------------------------------------------"""

def df_bu_split(bu):
    '''creates a seperate df for each bu'''
    df = das_df[(das_df['business_unit'] == bu)]
    df['business_unit'] = df['business_unit'].str.replace(' ', '_')
    
    return df  
    
for bu in bus_units:  
    '''to call a vertical...B830_CNET_Media'''
    globals()[str(bu)] = df_bu_split(bu)




def updated_bu_split(bu):
    '''creates a seperate df for each updated business unit '''
    df = das_df[(das_df['business_unit'] == bu)]
    
    return df 



for bu in bus_units:
    '''to call a vertical...B830_CNET_Media'''
    globals()[str(bu)] = updated_bu_split(bu)     




In [ ]:
#CNET_Marketing_Expenses#.groupby(['line_memo','supplier_as_worktag')

In [ ]:
def df_rev_site_pivot(df):
  df = pd.pivot_table(df, index = ['brand_region','revenue_type_rollup', 'channel_revcat_cat'], columns = ['month_dt','brand'], values = 'workday_rev_actuals', aggfunc= np.sum, margins = True, margins_name = 'Total', fill_value = 0).astype(int)

  return df

def df_rev_chan_pivot(df):
  df = pd.pivot_table(df, index = ['brand_region','ledger_account','ledger_account_rollup','revenue_type_rollup', 'channel_revcat_cat'], columns = ['month_dt'], values = 'workday_rev_actuals', aggfunc= np.sum, margins = True, margins_name = 'Total', fill_value = 0).astype(int)

  return df

def df_exp_pivot(df):
  df = pd.pivot_table(df, index = ['brand_region','ledger_account_rollup', 'brand'], columns = 'month_dt', values = 'workday_rev_actuals', aggfunc= np.sum, margins = True, margins_name = 'Total', fill_value = 0).astype(int)
  pd.set_option('display.float_format', '${:.0f}'.format)
  return df

def df_category_pivot(df):
  df = pd.pivot_table(df, index = ['category'], columns = 'month_dt', values = 'workday_rev_actuals', aggfunc= np.sum, margins = True, margins_name = 'Total', fill_value = 0).astype(int)

  return df  
#_______________________________________________________________________________________________________________  

def currency_total(df):
  '''creates a currency formatted total table by brand_region, revenue_rollup and channel concatenation'''
  #original
  df = pd.pivot_table(df, index = ['brand_region','revenue_type_rollup', 'channel_revcat_cat'], columns = ['month_dt','brand'], values = 'workday_rev_actuals', aggfunc= np.sum, fill_value = 0)#.astype(int)#.reset_index().drop(columns = [0], axis = 1)#.style.format({'workday_rev_actuals':"${0:,.0f}")

  
  
  df = df.reset_index()
  a = df.assign(channel_revcat_cat=lambda x: "total")
  b = df.assign(revenue_type_rollup=lambda x: "total", channel_revcat_cat	=lambda x: "total")
  c = df.assign(brand_region=lambda x: "total", revenue_type_rollup=lambda x: "total", channel_revcat_cat =lambda x: 'total')

  df = pd.concat([df,a, b, c]).groupby(['brand_region','revenue_type_rollup', 'channel_revcat_cat'], observed = True).sum()



# Drop last row 
# by selecting all rows except last row
  df = df.iloc[:-1 , :]



  pd.set_option('display.float_format', '${:,.0f}'.format)

  df= df.reset_index()

  #create a new dataframe filtered against the "Total" rows in the passed df, we will merge on index in later step
  col_tot = df[(df['channel_revcat_cat'] == 'total')].sum(axis = 1).to_frame().rename(columns = {0:'total'})

  df = pd.merge(df, col_tot, left_index= True, right_index = True, how = 'left')
  df = df.replace(np.nan,'', regex  = True)
  return df

#___________________________________________________________________________________________________
def updated_currency_total(df):

  '''createds a currency formatted table by updated rev type rollups against revenue categories'''
  df = pd.pivot_table(df, index = ['updated_rev_type_rollup', 'channel_revcat_cat'], columns = ['month_dt','brand'], values = 'workday_rev_actuals', aggfunc= np.sum, fill_value = 0).reset_index()
  
  a = df.assign(channel_revcat_cat=lambda x: "total")
  b = df.assign(updated_revenue_type_rollup=lambda x: "total", channel_revcat_cat	=lambda x: "total")

  df = pd.concat([df,a, b]).groupby(['updated_rev_type_rollup', 'channel_revcat_cat'], observed = True).sum().reset_index()

  # Drop last row 
  # by selecting all rows except last row
  #df = df.iloc[:-1 , :].reset_index()

  pd.set_option('display.float_format', '${:,.0f}'.format)

  

  #create a new dataframe filtered against the "Total" rows in the passed df, we will merge on index in later step
  col_tot = df[(df['channel_revcat_cat'] == 'total')].sum(axis = 1).to_frame().rename(columns = {0:'total'})

  df = pd.merge(df, col_tot, left_index= True, right_index = True, how = 'left')
  df = df.replace(np.nan,'', regex  = True)

  return df


#_____________________________________________________________________________________________________________
def updated_currency_total2(df):
  '''creates a currency formatted total table by brand_region, revenue_rollup and channel concatenation'''
  df = pd.pivot_table(df, index = ['updated_rev_type_rollup', 'channel_revcat_cat'], columns = ['month_dt','brand'], values = 'workday_rev_actuals', aggfunc= np.sum, fill_value = 0).reset_index()
  a = df.assign(channel_revcat_cat = lambda x: "total")
  b = df.assign(updated_rev_type_rollup = lambda x: "total",channel_revcat_cat = lambda x: "total")

  df = pd.concat([df,a,b]).groupby(['updated_rev_type_rollup', 'channel_revcat_cat'],observed = True).sum().reset_index()

  col_tot = df[(df['channel_revcat_cat'] == 'total')].sum(axis = 1).to_frame().rename(columns = {0:'total'})

  df = pd.merge(df,col_tot, left_index = True, right_index = True, how = 'left').replace(np.nan,"",regex = True)

  pd.set_option('display.float_format', '${:,.0f}'.format)

  return df


def updated_currency_total3(df, *args):

  '''creates a currency formatted total table by brand_region, revenue_rollup and channel concatenation'''
  #df = df[df['updated_rev_type_rollup']==type]

  
  df = df[df['updated_rev_type_rollup'].isin([*args])]

  df = pd.pivot_table(df, index = ['updated_rev_type_rollup', 'channel_revcat_cat', 'operative_order_id','opportunity_nm'], columns = ['month_dt','brand'], values = 'workday_rev_actuals', aggfunc= np.sum,margins = True, margins_name = 'Total', fill_value = 0).reset_index()
  #a = df.assign(channel_revcat_cat = lambda x: "total")
  #b = df.assign(updated_rev_type_rollup = lambda x: "total",channel_revcat_cat = lambda x: "total")
  #c = df.assign(operative_order_id = lambda x: "total",updated_rev_type_rollup = lambda x: "total",channel_revcat_cat = lambda x: "total")

  #df = pd.concat([df,a,b,c]).groupby(['updated_rev_type_rollup', 'channel_revcat_cat', 'operative_order_id'],observed = True).sum().reset_index()

  #col_tot = df[(df['channel_revcat_cat'] == 'total')].sum(axis = 1).to_frame().rename(columns = {0:'total'})

  #df = pd.merge(df,col_tot, left_index = True, right_index = True, how = 'left').replace(np.nan,"",regex = True)

  pd.set_option('display.float_format', '${:,.0f}'.format)
  pd.set_option('display.max_rows', None)

  return df


def updated_currency_total_id(df, *args):

  '''creates a currency formatted total table by brand_region, revenue_rollup and channel concatenation'''
  #df = df[df['updated_rev_type_rollup']==type]

  
  df = df[df['updated_rev_type_rollup'].isin([*args])]

  df = pd.pivot_table(df, index = [ 'operative_order_id','opportunity_nm'], columns = ['month_dt','brand'], values = 'workday_rev_actuals', aggfunc= np.sum,margins = True, margins_name = 'Total', fill_value = 0).reset_index()
 
  df = df.sort_values('Total', ascending = True)

  pd.set_option('display.float_format', '${:,.0f}'.format)
  pd.set_option('display.max_rows', None)

  return df



















#____________________________________________________________________________________________________________________
def df_exp_currency(df):
  df = pd.pivot_table(df, index = ['brand_region','ledger_account_rollup', 'brand'], columns = 'month_dt', values = 'workday_rev_actuals', aggfunc= np.sum, margins = True, margins_name = 'Total', fill_value = 0)#.astype(int)
  df = df.reset_index()
  pd.set_option('display.float_format', '${:,.0f}'.format)

  df =df.drop(['Total'], axis = 1)

  return df

#_____________________________________________________________________________________________________________
def updated_currency_total4(df):
  '''creates a currency formatted total table by brand_region, revenue_rollup and channel concatenation'''
  df = pd.pivot_table(df, index = ['updated_rev_type_rollup', 'channel_revcat_cat'], columns = ['month_dt','brand'], values = 'workday_rev_actuals', aggfunc= np.sum, fill_value = 0).reset_index()
  a = df.assign(channel_revcat_cat = lambda x: "total")
  b = df.assign(updated_rev_type_rollup = lambda x: "total",channel_revcat_cat = lambda x: "total")

  df = pd.concat([df,a,b]).groupby(['updated_rev_type_rollup', 'channel_revcat_cat','supplier_as_worktag'],observed = True).sum().reset_index()

  col_tot = df[(df['channel_revcat_cat'] == 'total')].sum(axis = 1).to_frame().rename(columns = {0:'total'})

  df = pd.merge(df,col_tot, left_index = True, right_index = True, how = 'left').replace(np.nan,"",regex = True)

  pd.set_option('display.float_format', '${:,.0f}'.format)

  return df  





In [ ]:

revenue_dfs = [CNET_Revenue,CNET_ROW_Revenue,B2B_Revenue,B2B_ROW_Revenue,Games_Revenue,Entertainment_Revenue,Other_Revenue]

#updated_revenue_dfs = [ CNET_Media_Revenue ,CNET_Tech_Revenue, CNET_Others_Revenue, ZD_Net_Revenue, Time_Revenue, Safety_Revenue,Product_Revenue, MSY_Revenue]#,   TR_Revenue,  Broadband_Revenue, Movers_Revenue, Energy_Revenue, Corporate_Revenue, ] #G&E(Fandom)_Revenue,G&E_Revenue,

updated_revenue_dfs = [CNET_Media_Revenue, ZD_Net_Revenue]
updated_combined = pd.concat([CNET_Media_Revenue, ZD_Net_Revenue])
updated_combined = [updated_combined]
                       
updated_bus_units = [B830_CNET_Media, B755_CNET_Technology, CNET_group_corp, B753_B2B___ZDNet, B548_TIME, B199_MySlumberYard_com, B930_Product]

"""expense_dfs = [
               CNET_Head_Count_Expenses,CNET_Marketing_Expenses,CNET_Non_Head_Count_Expense,CNET_Vertical_Allocation_Expense,CNET_Corporate_Allocation_Expense,
               CNET_ROW_Head_Count_Expenses,CNET_ROW_Marketing_Expenses,CNET_ROW_Non_Head_Count_Expense,CNET_ROW_Vertical_Allocation_Expense,CNET_ROW_Corporate_Allocation_Expense,

               B2B_Head_Count_Expenses,B2B_Marketing_Expenses,B2B_Non_Head_Count_Expense,B2B_Vertical_Allocation_Expense,B2B_Corporate_Allocation_Expense,
               B2B_ROW_Head_Count_Expenses,B2B_ROW_Marketing_Expenses,B2B_ROW_Non_Head_Count_Expense,B2B_ROW_Vertical_Allocation_Expense,B2B_ROW_Corporate_Allocation_Expense,

               Games_Head_Count_Expenses,Games_Marketing_Expenses,Games_Non_Head_Count_Expense,Games_Vertical_Allocation_Expense,Games_Corporate_Allocation_Expense,

               Entertainment_Head_Count_Expenses,Entertainment_Marketing_Expenses,Entertainment_Non_Head_Count_Expense,Entertainment_Vertical_Allocation_Expense,Entertainment_Corporate_Allocation_Expense,

               Other_Head_Count_Expenses,Other_Marketing_Expenses,Other_Non_Head_Count_Expense,Other_Vertical_Allocation_Expense,Other_Corporate_Allocation_Expense
               ]"""



'expense_dfs = [\n               CNET_Head_Count_Expenses,CNET_Marketing_Expenses,CNET_Non_Head_Count_Expense,CNET_Vertical_Allocation_Expense,CNET_Corporate_Allocation_Expense,\n               CNET_ROW_Head_Count_Expenses,CNET_ROW_Marketing_Expenses,CNET_ROW_Non_Head_Count_Expense,CNET_ROW_Vertical_Allocation_Expense,CNET_ROW_Corporate_Allocation_Expense,\n\n               B2B_Head_Count_Expenses,B2B_Marketing_Expenses,B2B_Non_Head_Count_Expense,B2B_Vertical_Allocation_Expense,B2B_Corporate_Allocation_Expense,\n               B2B_ROW_Head_Count_Expenses,B2B_ROW_Marketing_Expenses,B2B_ROW_Non_Head_Count_Expense,B2B_ROW_Vertical_Allocation_Expense,B2B_ROW_Corporate_Allocation_Expense,\n\n               Games_Head_Count_Expenses,Games_Marketing_Expenses,Games_Non_Head_Count_Expense,Games_Vertical_Allocation_Expense,Games_Corporate_Allocation_Expense,\n\n               Entertainment_Head_Count_Expenses,Entertainment_Marketing_Expenses,Entertainment_Non_Head_Count_Expense,Entertainment_Ver

# Revenue Data Frames

In [ ]:
pd.set_option('display.max_columns', None)
#das_df[das_df['operative_order_id']=='80709']
CNET_Media[CNET_Media['channel_revcat_cat']=='C010 Production Fees _ Other']
#CNET_Media.opportunity_nm.isnull()
           
           #=='IO 82471 - APR PROVISION 14%']


,day_dt,month_dt,quarter_dt,year_dt,company,region,journal_number,journal_name,ledger_status,ledger_source,ledger_type,operational_transaction,line_memo,operative_order_id,channel_revcat_cat,revenue_type_rollup,business_unit,brand,brand_region,ledger_account,ledger_account_rollup,customer,cost_center,category,opportunity_nm,workday_rev_actuals,supplier_as_worktag,spend_category_as_worktag,spend_cat_id,exclusion_filter,updated_bu_map,updated_rev_type_rollup,expense_flag,business_unit_original
3239,2023-04-30,4,2,2023,"RV025 CNET Media, Inc",US,None,"Operational Journal: RV025 CNET Media, Inc - 0...",Posted,Customer Invoice,Actuals,Customer Invoice: CM-207749,81329CM|a||CNET|Syndicated Content||Fee|||||||...,81329,C010 Production Fees _ Other,Direct Ad Sales,B830_CNET_Media,CNET Media,CNET,4005:Other Revenue,Revenue,SnapChat - CNET,D700 Other - Sales Support,None,blank,-478.80,(blank),None,None,RV,CNET_Media,DAS_Onsite_Rev,4005:Other Revenue,B830 CNET Media
63771,2023-04-30,4,2,2023,"RV025 CNET Media, Inc",US,RV025-JE_00003718,"RV025-JE_00003718 - RV025 CNET Media, Inc - 04...",Posted,Manual Journal,Actuals,None,IO# 80156 - Invoice 206970 - Accrual April-2023,80156,C010 Production Fees _ Other,Direct Ad Sales,B830_CNET_Media,CNET Media,CNET,1102:Accrued Accounts Receivable,1102:Accrued_Accounts_Receivable,Facebook Alpha Product,D700 Other - Sales Support,Incubator,blank,4143.99,(blank),None,None,RV,CNET_Media,DAS_Onsite_Rev,1102:Accrued Accounts Receivable,B830 CNET Media
63778,2023-04-30,4,2,2023,"RV025 CNET Media, Inc",US,RV025-JE_00003718,"RV025-JE_00003718 - RV025 CNET Media, Inc - 04...",Posted,Manual Journal,Actuals,None,IO# 80156 - Invoice 206970 - Accrual April-2023,80156,C010 Production Fees _ Other,Direct Ad Sales,B830_CNET_Media,CNET Media,CNET,4005:Other Revenue,Revenue,Facebook Alpha Product,D700 Other - Sales Support,Incubator,blank,-4143.99,(blank),None,None,RV,CNET_Media,DAS_Onsite_Rev,4005:Other Revenue,B830 CNET Media


In [ ]:
for df in updated_revenue_dfs:
  display(updated_currency_total2(df))
  print('\n')
  print('\n')
  

,"(updated_rev_type_rollup, )","(channel_revcat_cat, )","(4, CNET Media)",total
0,CNET_Media,C265 Display _ Licensing,$0,
1,CNET_Media,C594 Partners _ Other,"$7,500",
2,CNET_Media,total,"$7,500","$7,500"
3,DAS_Offsite_Backfill_Rev(YT),C120 Video - Offsite _ Licensing,"$-27,539",
4,DAS_Offsite_Backfill_Rev(YT),C120 Video - Offsite _ Video Exchange,"$-46,900",
5,DAS_Offsite_Backfill_Rev(YT),total,"$-74,439","$-74,439"
6,DAS_Offsite_Rev(Social),C957 Social Media _ Other,"$-10,872",
7,DAS_Offsite_Rev(Social),total,"$-10,872","$-10,872"
8,DAS_Onsite_Backfill_Rev(OE),C002 Audio _ Desktop Exchange,"$2,015",
9,DAS_Onsite_Backfill_Rev(OE),C500 Programmatic _ Desktop,"$-6,294",


,"(updated_rev_type_rollup, )","(channel_revcat_cat, )","(4, ZD Net)",total
0,DAS_Offsite_Rev(Social),C957 Social Media _ Other,"$-4,340",
1,DAS_Offsite_Rev(Social),total,"$-4,340","$-4,340"
2,DAS_Onsite_Backfill_Rev(OE),C002 Audio _ Desktop Exchange,$-5,
3,DAS_Onsite_Backfill_Rev(OE),C500 Programmatic _ Desktop,$-77,
4,DAS_Onsite_Backfill_Rev(OE),C500 Programmatic _ Desktop Exchange,"$-33,199",
5,DAS_Onsite_Backfill_Rev(OE),C500 Programmatic _ Mobile Exchange,"$-55,031",
6,DAS_Onsite_Backfill_Rev(OE),C500 Programmatic _ Video Exchange,"$-2,983",
7,DAS_Onsite_Backfill_Rev(OE),C726 Email/Co-Reg _ Desktop Exchange,"$-1,108",
8,DAS_Onsite_Backfill_Rev(OE),total,"$-92,404","$-92,404"
9,DAS_Onsite_Backfill_Rev(PMP),Programmatic PMP _ Desktop Exchange,"$-7,009",


In [ ]:
for df in updated_combined:
  display(updated_currency_total3(df,'DAS_Onsite_Rev','DAS_Offsite_Rev(Social)','Email_Media_Rev',))
  print('\n')
  print('\n')

month_dt  updated_rev_type_rollup  \
brand                               
0         DAS_Offsite_Rev(Social)   
1         DAS_Offsite_Rev(Social)   
2         DAS_Offsite_Rev(Social)   
3         DAS_Offsite_Rev(Social)   
4         DAS_Offsite_Rev(Social)   
5         DAS_Offsite_Rev(Social)   
6                  DAS_Onsite_Rev   
7                  DAS_Onsite_Rev   
8                  DAS_Onsite_Rev   
9                  DAS_Onsite_Rev   
10                 DAS_Onsite_Rev   
11                 DAS_Onsite_Rev   
12                 DAS_Onsite_Rev   
13                 DAS_Onsite_Rev   
14                 DAS_Onsite_Rev   
15                 DAS_Onsite_Rev   
16                 DAS_Onsite_Rev   
17                 DAS_Onsite_Rev   
18                 DAS_Onsite_Rev   
19                 DAS_Onsite_Rev   
20                 DAS_Onsite_Rev   
21                 DAS_Onsite_Rev   
22                 DAS_Onsite_Rev   
23                 DAS_Onsite_Rev   
24                 DAS_Onsite_Rev   
25                 DAS_Onsite_Rev   
26                 DAS_Onsite_Rev   
27                 DAS_Onsite_Rev   
28                 DAS_Onsite_Rev   
29                 DAS_Onsite_Rev   
30                 DAS_Onsite_Rev   
31                 DAS_Onsite_Rev   
32                 DAS_Onsite_Rev   
33                 DAS_Onsite_Rev   
34                 DAS_Onsite_Rev   
35                 DAS_Onsite_Rev   
36                 DAS_Onsite_Rev   
37                 DAS_Onsite_Rev   
38                 DAS_Onsite_Rev   
39                 DAS_Onsite_Rev   
40                 DAS_Onsite_Rev   
41                 DAS_Onsite_Rev   
42                 DAS_Onsite_Rev   
43                 DAS_Onsite_Rev   
44                 DAS_Onsite_Rev   
45                 DAS_Onsite_Rev   
46                 DAS_Onsite_Rev   
47                 DAS_Onsite_Rev   
48                 DAS_Onsite_Rev   
49                 DAS_Onsite_Rev   
50                 DAS_Onsite_Rev   
51                 DAS_Onsite_Rev   
52                 DAS_Onsite_Rev   
53                 DAS_Onsite_Rev   
54                 DAS_Onsite_Rev   
55                Email_Media_Rev   
56                Email_Media_Rev   
57                          Total   

month_dt                                 channel_revcat_cat  \
brand                                                         
0                                 C957 Social Media _ Other   
1                                 C957 Social Media _ Other   
2                                 C957 Social Media _ Other   
3                                 C957 Social Media _ Other   
4                                 C957 Social Media _ Other   
5                                 C957 Social Media _ Other   
6                              C010 Production Fees _ Other   
7                              C010 Production Fees _ Other   
8                               C121 Video - Onsite _ Video   
9                               C121 Video - Onsite _ Video   
10                              C121 Video - Onsite _ Video   
11                              C121 Video - Onsite _ Video   
12                              C121 Video - Onsite _ Video   
13                              C121 Video - Onsite _ Video   
14                              C121 Video - Onsite _ Video   
15                              C121 Video - Onsite _ Video   
16                              C121 Video - Onsite _ Video   
17                              C121 Video - Onsite _ Video   
18                              C121 Video - Onsite _ Video   
19                                   C265 Display _ Desktop   
20                                   C265 Display _ Desktop   
21                                   C265 Display _ Desktop   
22                                   C265 Display _ Desktop   
23                                   C265 Display _ Desktop   
24                                   C265 Display _ Desktop   
25                                   C265 Display _ Desktop   
26             

In [ ]:
for df in updated_combined:
  display(updated_currency_total_id(df,'DAS_Onsite_Rev','DAS_Offsite_Rev(Social)','Email_Media_Rev'))

month_dt operative_order_id  \
brand                         
44                    Total   
39                    82600   
5                     82239   
2                     80709   
7                     82452   
32                    82585   
23                    82554   
36                    82595   
24                    82558   
22                    82552   
41                    82611   
26                    82570   
14                    82502   
9                     82477   
8                     82471   
35                    82593   
17                    82510   
29                    82578   
30                    82580   
21                    82533   
33                    82587   
28                    82575   
40                    82609   
38                    82599   
37                    82597   
1                     80156   
27                    82571   
6                     82446   
15                    82507   
25                    82564   
3                     81329   
10                    82488   
31                    82581   
12                    82494   
19                    82518   
34                    82589   
20                    82522   
11                    82491   
0                     80034   
13                    82495   
16                    82509   
18                    82516   
42                    IO 82   
43                    Mover   
4                     81925   

month_dt                                     opportunity_nm          4  \
brand                                                       CNET Media   
44                                                           $-643,017   
39                               ViacomCBS_Premieres_Q22023  $-150,000   
5         Boost Mobile - SIM Media Strategy (Articles/So...  $-102,288   
2                              47712_Google_AdX_RevShare_PG   $-75,490   
7                          HPE_FY23_Annual Unified Campaign   $-15,466   
32                                Samsung_CTV Premium_Q2'23   $-53,622   
23                         Cisco Thought Leadership FY23 1H         $0   
36                                          TMobile_HINT_Q2   $-43,522   
24                    Nutanix_FY23Q3_Global Brand AMS_ZDNet         $0   
22               Cisco Meraki_CNET & ZDNet Partnership 2023   $-11,163   
41                                 Wayfair - WayDay (April)   $-25,000   
26                Nutanix_FY23Q3_Global Brand EMEA_ZDNet UK         $0   
14                NFCU_23_UMA_BRND_74056_DIS_General Market   $-23,014   
9                                      Bitdefender_Q2/Q3'23   $-22,000   
8         2023|Q1Q2Q3Q4|IBM|MKT|RTNR|NA|BTD|SiteDirect|4...   $-13,438   
35                                 Intel_Arc_Affiliate_2023         $0   
17        COMN_DIVR_74914_GM_BASE_23_1234_AWA_MMAW_Mid_F...   $-16,716   
29             CS_Algolia_Q1_FY24_B2C Ecommerce_RedVentures   $-10,624   
30                                            Instacart SMB    $-7,036   
21                         Bitwarden_Q1 FY23_Custom Article    $-4,578   
33                                 Cisco_Meraki Go_1Q-2Q'23    $-8,830   
28                                         Upside - Q1 2023   $-10,872   
40                                         Oura_Q2'23 Video    $-8,935   
38                                    Home Chef_Video_Q2'23    $-8,817   
37                                        Udemy_Video_Q2'23    $-6,919   
1                                                     blank    $-4,144   
27                    Chase United | Club Card RFP Q1/Q2'23    $-2,484   
6                         FY23_Siemens AG_Employer Branding      $-955   
15                                         Oura_Video_Q1'23      $-608   
25                   BBUY_COMP_Q12023_FY24 Q1 Intel Refresh   $-19,248   
3                                                     blank      $-479   
10                                        Udemy_Video_Q1'23      $-137   
31                        

In [ ]:
#for df in revenue_dfs:
  #OLD 2022 FORMAT
  
  #display(currency_total(df))
  #print('\n')
  #print('\n')

In [ ]:
#for df in expense_dfs: 
 # try:
    
  #  display(df_exp_currency(df))
   # print('\n','\n') 
    #pd.set_option('display.float_format', '${:,.2f}'.format)
      

  #several of the expense df's do not have expenses tied to them... therefore loop will break.  Try and except implemented
  #except:
   # pass 
  
  



  

# Exp: Budget Comparison

In [ ]:
import gspread
from gspread_dataframe import set_with_dataframe
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)



def df_format(df):
  '''changes dataframe format for comparison against actual'''
  headers = df.pop(0)
  df = pd.DataFrame(df, columns = headers).rename(columns = {'SUM of 1/1/2023':'Jan_budg','SUM of 2/1/2023':'Feb_budg','SUM of 3/1/2023':'Mar_budg','SUM of 4/1/2023':'Apr_budg','SUM of 5/1/2023':'May_budg','SUM of 6/1/2023':'Jun_budg',
                                                                    'SUM of 7/1/2023':'Jul_budg','SUM of 8/1/2023':'Aug_budg','SUM of 9/1/2023':'Sep_budg','SUM of 10/1/2023':'Oct_budg','SUM of 11/1/2023':'Nov_budg','SUM of 12/1/2023':'Dec_budg','BU':'business_unit'})
  
  df[['Jan_budg','Feb_budg','Mar_budg','Apr_budg','May_budg','Jun_budg','Jul_budg','Aug_budg','Sep_budg','Oct_budg','Nov_budg','Dec_budg']] = df[['Jan_budg','Feb_budg','Mar_budg','Apr_budg','May_budg','Jun_budg','Jul_budg','Aug_budg','Sep_budg','Oct_budg','Nov_budg','Dec_budg']].apply(pd.to_numeric)  

  return df

def df_filter(df,*args):

  'df filter where we can filter for specific multiple column names, ex.) Jan or BU'
  df= df[[*args]]
  return df


 #new expense df 
exp = das_df[(das_df['expense_flag'] == 'm&c_hc_expenses')]  

#GRABBING sheets data ,altering df formats and time periods

nhc_budg = gc.open('2023_non_hc_v2_A.J.')

all_piv = nhc_budg.worksheet('non_hc_budget_pivot').get('A1:P1312')
all_piv = df_filter(df_format(all_piv),"business_unit", 'spend_category_as_worktag','supplier_as_worktag','Mar_budg')

bu_piv = nhc_budg.worksheet('bu_non_hc_budget_pivot').get('A1:M7')
#change budget month name per month
bu_piv = df_filter(df_format(bu_piv),'business_unit','Mar_budg')


spend_piv = nhc_budg.worksheet('spend_cat_non_hc_budget_pivot').get('A1:N196')
# change budget month name per month
spend_piv = df_filter(df_format(spend_piv),'business_unit','spend_category_as_worktag','Mar_budg')


supp_piv = nhc_budg.worksheet('supplier_non_hc_budget_pivot').get('A1:N943')
# change budget month name per month
supp_piv = df_filter(df_format(supp_piv),'business_unit','supplier_as_worktag','Mar_budg')


# Business Unit
bus_exps = pd.pivot_table(exp, index = ['business_unit'], columns = ['month_dt'], values = 'workday_rev_actuals', aggfunc= np.sum, fill_value = 0).reset_index()


#spend category
spend_exps = pd.pivot_table(exp, index = ['business_unit','spend_category_as_worktag'], columns = ['month_dt'], values = 'workday_rev_actuals', aggfunc= np.sum, fill_value = 0).reset_index()

#Supplier As Worktag
supp_exps =pd.pivot_table(exp, index = ['business_unit','supplier_as_worktag'], columns = ['month_dt'], values = 'workday_rev_actuals', aggfunc= np.sum, fill_value = 0).reset_index()  

#All pivot - bu, spend, supplier
all_exps =pd.pivot_table(exp, index = ['business_unit','spend_category_as_worktag','supplier_as_worktag'], columns = ['month_dt'], values = 'workday_rev_actuals', aggfunc= np.sum, fill_value = 0).reset_index()  





#individual business unit worksheets
cnet_media_ws = nhc_budg.worksheet('cnet_media')

cnet_tech_ws = nhc_budg.worksheet('cnet_tech')

cnet_corp_ws = nhc_budg.worksheet('cnet_corp')

zdnet_ws = nhc_budg.worksheet('zd_net')

time_ws = nhc_budg.worksheet('time')

msy_ws = nhc_budg.worksheet('msy')

product_ws = nhc_budg.worksheet('product')








In [ ]:
#exp[exp['business_unit'].eq('B753_B2B___ZDNet')]
pd.set_option('display.max_columns', None)
exp[(exp['business_unit']== 'B753_B2B___ZDNet') & (exp['spend_category_as_worktag'].isin(['Software Licenses / SaaS']))]

,day_dt,month_dt,quarter_dt,year_dt,company,region,journal_number,journal_name,ledger_status,ledger_source,ledger_type,operational_transaction,line_memo,operative_order_id,channel_revcat_cat,revenue_type_rollup,business_unit,brand,brand_region,ledger_account,ledger_account_rollup,customer,cost_center,category,opportunity_nm,workday_rev_actuals,supplier_as_worktag,spend_category_as_worktag,spend_cat_id,exclusion_filter,updated_bu_map,updated_rev_type_rollup,expense_flag,business_unit_original
9950,2023-04-01,4,2,2023,"RV025 CNET Media, Inc",US,None,"Operational Journal: RV025 CNET Media, Inc - 0...",Posted,Supplier Invoice,Actuals,Supplier Invoice: 1197830,None,None,None,None,B753_B2B___ZDNet,ZD Net,B2B,7300:Technology,Non_Head_Count_Expense,None,D790 IT Operations,None,blank,$214,LiveIntent (RV),Software Licenses / SaaS,SC115,RV,ZD_Net,ZD_Net,m&c_hc_expenses,B753 B2B - ZDNet
9971,2023-04-01,4,2,2023,"RV025 CNET Media, Inc",US,None,"Operational Journal: RV025 CNET Media, Inc - 0...",Posted,Supplier Invoice,Actuals,Supplier Invoice: 1194536,None,None,None,None,B753_B2B___ZDNet,ZD Net,B2B,7300:Technology,Non_Head_Count_Expense,None,D790 IT Operations,None,blank,$88,LiveIntent (RV),Software Licenses / SaaS,SC115,RV,ZD_Net,ZD_Net,m&c_hc_expenses,B753 B2B - ZDNet
10230,2023-04-28,4,2,2023,"RV025 CNET Media, Inc",US,RV025-JE_00003755,"RV025-JE_00003755 - RV025 CNET Media, Inc - 04...",Posted,Red Ventures Spreadsheet Import,Actuals,None,Release Journal - March 23 - Accrual Jrnl Mar ...,Relea,None,None,B753_B2B___ZDNet,ZD Net,B2B,7300:Technology,Non_Head_Count_Expense,None,D790 IT Operations,None,blank,$0,LiveIntent (RV),Software Licenses / SaaS,SC115,RV,ZD_Net,ZD_Net,m&c_hc_expenses,B753 B2B - ZDNet
68576,2023-04-01,4,2,2023,"RV025 CNET Media, Inc",US,None,"Operational Journal: RV025 CNET Media, Inc - 0...",Posted,Prepaid Spend Amortization,Actuals,Amortization Prepaid Spend: Supplier Invoice: ...,None,None,None,None,B753_B2B___ZDNet,ZD Net,B2B,7300:Technology,Non_Head_Count_Expense,None,D700 Other - Sales Support,None,blank,"$1,350",TRUE ANTHEM CORPORATION,Software Licenses / SaaS,SC115,RV,ZD_Net,ZD_Net,m&c_hc_expenses,B753 B2B - ZDNet
68867,2023-04-28,4,2,2023,"RV025 CNET Media, Inc",US,RV025-JE_00003755,"RV025-JE_00003755 - RV025 CNET Media, Inc - 04...",Posted,Red Ventures Spreadsheet Import,Actuals,None,Release Journal - March 23 - Accrual Jrnl Mar ...,Relea,None,None,B753_B2B___ZDNet,ZD Net,B2B,7300:Technology,Non_Head_Count_Expense,None,D790 IT Operations,None,blank,$0,LiveIntent (RV),Software Licenses / SaaS,SC115,RV,ZD_Net,ZD_Net,m&c_hc_expenses,B753 B2B - ZDNet
68884,2023-04-30,4,2,2023,"RV025 CNET Media, Inc",US,RV025-JE_00003761,"RV025-JE_00003761 - RV025 CNET Media, Inc - 04...",Posted,Manual Journal,Actuals,None,MAPP Digital correction - INV-4210737 Jan 2023,MAPP,None,None,B753_B2B___ZDNet,ZD Net,B2B,7300:Technology,Non_Head_Count_Expense,None,D710 Business Ops,None,blank,"$5,248",MAPP DIGITAL US LLC,Software Licenses / SaaS,SC115,RV,ZD_Net,ZD_Net,m&c_hc_expenses,B753 B2B - ZDNet


In [ ]:
xx
#delete_range = cnet_media_ws.range("A2:AB10000")
#cnet_media_ws.values_clear(delete_range)


NameError: ignored

## CNET Media

In [ ]:
for df in updated_bus_units:
  if id(df) in [id(B830_CNET_Media)]:
   

    #new expense df for the passed business units
    #exp = df[(df['expense_flag'] == 'm&c_expenses')]
    bu_piv_media = bu_piv[(bu_piv['business_unit']=='B830_CNET_Media')]
    spend_piv_media = spend_piv[(spend_piv['business_unit']=='B830_CNET_Media')]
    supp_piv_media = supp_piv[(supp_piv['business_unit']=='B830_CNET_Media')] 
    all_piv_media = all_piv[(all_piv['business_unit']=='B830_CNET_Media')]
       
     # Business Unit
    #bus_exps = pd.pivot_table(exp, index = ['business_unit'], columns = ['month_dt'], values = 'workday_rev_actuals', aggfunc= np.sum, fill_value = 0).reset_index()
    #display(bus_exps)
    #print('\n')

    bus_exps_media = pd.merge(bus_exps, bu_piv_media, left_on = 'business_unit', right_on ='business_unit', how = 'right').replace(np.nan,"",regex = True)
    bus_exps_media = bus_exps_media[(bus_exps_media['business_unit']=='B830_CNET_Media')]
    #display(bus_exps)
    #bu_wksht = cnet_media_ws

    print('Do you want to update the CNET Media sheet with new BUSINESS UNIT data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What business unit row #(only numerical values)?')
      row = int(input())
      print('What business unit column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_media_ws,bus_exps_media,row,column)
      print('new data within the CNET Media tab has been populated with the coordinate parameters given')
      print('\n')
  
    elif(response == 'Yes'):
      print('What business unit row #(only numerical values)?')
      row = int(input())
      print('What business unit column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_media_ws,bus_exps_media,row,column)
      print('new data within the CNET Media tab has been populated with the coordinate parameters given')
      print('\n')

    else:
      print('Nothing to do, moving on, have a nice day')
      print('\n')

  #spend category
    #spend_exps = pd.pivot_table(exp, index = ['business_unit','spend_category_as_worktag'], columns = ['month_dt'], values = 'workday_rev_actuals', aggfunc= np.sum, fill_value = 0).reset_index()
 
    spend_exps_media = pd.merge(spend_exps, spend_piv_media, left_on = ['business_unit','spend_category_as_worktag'], right_on = ['business_unit','spend_category_as_worktag'], how = 'outer').replace(np.nan,"",regex = True)
    spend_exps_media = spend_exps_media[(spend_exps_media['business_unit']=='B830_CNET_Media')]

    print('Do you want to update the CNET Media sheet with new SPEND CATEGORY data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What spend category row #(only numerical values)?')
      row = int(input())
      print('What spend category column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_media_ws,spend_exps_media,row,column)
      print('new data has been populated in coordinate parameters given')
      print('\n')
  
    elif(response == 'Yes'):
      print('What spend category row #(only numerical values)?')
      row = int(input())
      print('What column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_media_ws,spend_exps_media,row,column)
      print('new data has been populated in coordinate parameters given')
      print('\n')

    else:
      print('Nothing to do, moving on, have a nice day')
      print('\n')

  #Supplier As Worktag
    #supp_exps =pd.pivot_table(exp, index = ['business_unit','supplier_as_worktag'], columns = ['month_dt'], values = 'workday_rev_actuals', aggfunc= np.sum, fill_value = 0).reset_index()  
    supp_exps_media = pd.merge(supp_exps, supp_piv_media, left_on = ['business_unit', 'supplier_as_worktag'], right_on =['business_unit','supplier_as_worktag'], how = 'outer').replace(np.nan,"",regex = True)
    supp_exps_media = supp_exps_media[(supp_exps_media['business_unit']=='B830_CNET_Media')]
    


    print('Do you want to update the CNET Media sheet with SUPPLIER SPEND data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What supplier spend row #(only numerical values)?')
      row = int(input())
      print('What supplier spend column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_media_ws,supp_exps_media,row,column)
      print('new data has been populated in coordinate parameters given')
      print('\n')
  
    elif(response == 'Yes'):
      print('What supplier spend row #(only numerical values)?')
      row = int(input())
      print('What supplier spend column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_media_ws,supp_exps_media,row,column)
      print('new data has been populated in coordinate parameters given')
      print('\n')


    else:
      print('Nothing to do, moving on, have a nice day')  
      print('\n')     

  #All exps      
    all_exps_media = pd.merge(all_exps, all_piv_media, left_on = ['business_unit', 'spend_category_as_worktag','supplier_as_worktag'], right_on =['business_unit', 'spend_category_as_worktag','supplier_as_worktag'], how = 'outer').replace(np.nan,"",regex = True)
    all_exps_media = all_exps_media[(all_exps_media['business_unit']=='B830_CNET_Media')]
    


    print('Do you want to update the CNET Media sheet with ALL EXPENSE data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What all expense row #(only numerical values)?')
      row = int(input())
      print('What all expense column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_media_ws,all_exps_media,row,column)
      print('new data has been populated in coordinate parameters given')
      print('\n')
  
    elif(response == 'Yes'):
      print('What all expsnse row #(only numerical values)?')
      row = int(input())
      print('What all expense column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_media_ws,all_exps_media,row,column)
      print('new data has been populated in coordinate parameters given, and we are done here')
      print('\n')


    else:
      print('Nothing to do, moving on, have a nice day')  
      print('\n')


      


## CNET Tech

In [ ]:
for df in updated_bus_units:
  if id(df) in [id(B755_CNET_Technology)]:



    #new expense df for the passed business units
    #exp = df[(df['expense_flag'] == 'm&c_expenses')]
    bu_piv_tech = bu_piv[(bu_piv['business_unit']=='B755_CNET_Technology')]
    spend_piv_tech = spend_piv[(spend_piv['business_unit']=='B755_CNET_Technology')]
    supp_piv_tech = supp_piv[(supp_piv['business_unit']=='B755_CNET_Technology')] 
    all_piv_tech = all_piv[(all_piv['business_unit']=='B755_CNET_Technology')]
       
     # Business Unit
    #bus_exps = pd.pivot_table(exp, index = ['business_unit'], columns = ['month_dt'], values = 'workday_rev_actuals', aggfunc= np.sum, fill_value = 0).reset_index()
    #display(bus_exps)
    #print('\n')

    bus_exps_tech = pd.merge(bus_exps, bu_piv_tech, left_on = 'business_unit', right_on ='business_unit', how = 'right').replace(np.nan,"",regex = True)
    bus_exps_tech = bus_exps_tech[(bus_exps_tech['business_unit']=='B755_CNET_Technology')]
  
   

    print('Do you want to update the CNET Tech sheet with new BUSINESS UNIT data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What business unit row #(only numerical values)?')
      row = int(input())
      print('What business unit column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_tech_ws,bus_exps_tech,row,column)
      print('new data within the CNET Tech tab has been populated with the coordinate parameters given')
      print('\n')
  
    elif(response == 'Yes'):
      print('What business unit row #(only numerical values)?')
      row = int(input())
      print('What business unit column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_tech_ws,bus_exps_tech,row,column)
      print('new data within the CNET Tech tab has been populated with the coordinate parameters given')
      print('\n')

    else:
      print('Nothing to do, moving on, have a nice day')
      print('\n')

  #spend category
    #spend_exps = pd.pivot_table(exp, index = ['business_unit','spend_category_as_worktag'], columns = ['month_dt'], values = 'workday_rev_actuals', aggfunc= np.sum, fill_value = 0).reset_index() 
    spend_exps_tech = pd.merge(spend_exps, spend_piv_tech, left_on = ['business_unit','spend_category_as_worktag'], right_on = ['business_unit','spend_category_as_worktag'], how = 'outer').replace(np.nan,"",regex = True)
    spend_exps_tech = spend_exps_tech[(spend_exps_tech['business_unit']=='B755_CNET_Technology')]

    print('Do you want to update the CNET Tech sheet with new SPEND CATEGORY data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What spend category row #(only numerical values)?')
      row = int(input())
      print('What spend category column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_tech_ws,spend_exps_tech,row,column)
      print('new data has been populated in coordinate parameters given')
      print('\n')
  
    elif(response == 'Yes'):
      print('What spend category row #(only numerical values)?')
      row = int(input())
      print('What column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_tech_ws,spend_exps_tech,row,column)
      print('new data has been populated in coordinate parameters given')
      print('\n')

    else:
      print('Nothing to do, moving on, have a nice day')
      print('\n')

  #Supplier As Worktag
    #supp_exps =pd.pivot_table(exp, index = ['business_unit','supplier_as_worktag'], columns = ['month_dt'], values = 'workday_rev_actuals', aggfunc= np.sum, fill_value = 0).reset_index()  
    supp_exps_tech = pd.merge(supp_exps, supp_piv_tech, left_on = ['business_unit', 'supplier_as_worktag'], right_on =['business_unit','supplier_as_worktag'], how = 'outer').replace(np.nan,"",regex = True)
    supp_exps_tech = supp_exps_tech[(supp_exps_tech['business_unit']=='B755_CNET_Technology')]


    print('Do you want to update the CNET Tech sheet with SUPPLIER SPEND data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What supplier spend row #(only numberical values)?')
      row = int(input())
      print('What supplier spend column #(only numberical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_tech_ws,supp_exps_tech,row,column)
      print('new data has been populated in coordinate parameters given, and we are done here')
      print('\n')
  
    elif(response == 'Yes'):
      print('What supplier spend row #(only numerical values)?')
      row = int(input())
      print('What supplier spend column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_tech_ws,supp_exps_tech,row,column)
      print('new data has been populated in coordinate parameters given, and we are done here')
      print('\n')


    else:
      print('Nothing to do, moving on, have a nice day')  
      print('\n') 



  #All exps      
    all_exps_tech = pd.merge(all_exps, all_piv_tech, left_on = ['business_unit', 'spend_category_as_worktag','supplier_as_worktag'], right_on =['business_unit', 'spend_category_as_worktag','supplier_as_worktag'], how = 'outer').replace(np.nan,"",regex = True)
    all_exps_tech = all_exps_tech[(all_exps_tech['business_unit']=='B755_CNET_Technology')]
    


    print('Do you want to update the CNET Tech sheet with ALL EXPENSE data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What all expense row #(only numerical values)?')
      row = int(input())
      print('What all expense column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_tech_ws,all_exps_tech,row,column)
      print('new data has been populated in coordinate parameters given')
      print('\n')
  
    elif(response == 'Yes'):
      print('What all expsnse row #(only numerical values)?')
      row = int(input())
      print('What all expense column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_tech_ws,all_exps_tech,row,column)
      print('new data has been populated in coordinate parameters given, and we are done here')
      print('\n')


    else:
      print('Nothing to do, moving on, have a nice day')  
      print('\n')      

## CNET Corporate

In [ ]:
for df in updated_bus_units:
  if id(df) in [id(CNET_group_corp)]:



    #new expense df for the passed business units
    #exp = df[(df['expense_flag'] == 'm&c_expenses')]
    bu_piv_corp = bu_piv[(bu_piv['business_unit']=='CNET_group_corp')]
    spend_piv_corp = spend_piv[(spend_piv['business_unit']=='CNET_group_corp')]
    supp_piv_corp = supp_piv[(supp_piv['business_unit']=='CNET_group_corp')]
    all_piv_corp = all_piv[(all_piv['business_unit']=='CNET_group_corp')] 
       
     # Business Unit
    bus_exps_corp = pd.merge(bus_exps, bu_piv_corp, left_on = 'business_unit', right_on ='business_unit', how = 'right').replace(np.nan,"",regex = True)
    bus_exps_corp = bus_exps_corp[(bus_exps_corp['business_unit']=='CNET_group_corp')]

    print('Do you want to update the CNET Corp sheet with new BUSINESS UNIT data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What business unit row #(only numerical values)?')
      row = int(input())
      print('What business unit column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_corp_ws,bus_exps_corp,row,column)
      print('new data within the CNET Tech tab has been populated with the coordinate parameters given')
      print('\n')
  
    elif(response == 'Yes'):
      print('What business unit row #(only numerical values)?')
      row = int(input())
      print('What business unit column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_corp_ws,bus_exps_corp,row,column)
      print('new data within the CNET Tech tab has been populated with the coordinate parameters given')
      print('\n')

    else:
      print('Nothing to do, moving on, have a nice day')
      print('\n')

  #spend category
    spend_exps_corp = pd.merge(spend_exps, spend_piv_corp, left_on = ['business_unit','spend_category_as_worktag'], right_on = ['business_unit','spend_category_as_worktag'], how = 'outer').replace(np.nan,"",regex = True)
    spend_exps_corp = spend_exps_corp[(spend_exps_corp['business_unit']=='CNET_group_corp')]

    print('Do you want to update the CNET Corp sheet with new SPEND CATEGORY data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What spend category row #(only numerical values)?')
      row = int(input())
      print('What spend category column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_corp_ws,spend_exps_corp,row,column)
      print('new data has been populated in coordinate parameters given')
      print('\n')
  
    elif(response == 'Yes'):
      print('What spend category row #(only numerical values)?')
      row = int(input())
      print('What column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_corp_ws,spend_exps_corp,row,column)
      print('new data has been populated in coordinate parameters given')
      print('\n')

    else:
      print('Nothing to do, moving on, have a nice day')
      print('\n')

  #Supplier As Worktag
    #supp_exps =pd.pivot_table(exp, index = ['business_unit','supplier_as_worktag'], columns = ['month_dt'], values = 'workday_rev_actuals', aggfunc= np.sum, fill_value = 0).reset_index()  
    supp_exps_corp = pd.merge(supp_exps, supp_piv_corp, left_on = ['business_unit', 'supplier_as_worktag'], right_on =['business_unit','supplier_as_worktag'], how = 'outer').replace(np.nan,"",regex = True)
    supp_exps_corp = supp_exps_corp[(supp_exps_corp['business_unit']=='CNET_group_corp')]


    print('Do you want to update the CNET Corp sheet with SUPPLIER SPEND data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What supplier spend row #(only numberical values)?')
      row = int(input())
      print('What supplier spend column #(only numberical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_corp_ws,supp_exps_corp,row,column)
      print('new data has been populated in coordinate parameters given, and we are done here')
      print('\n')
  
    elif(response == 'Yes'):
      print('What supplier spend row #(only numberical values)?')
      row = int(input())
      print('What supplier spend column #(only numberical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_corp_ws,supp_exps_corp,row,column)
      print('new data has been populated in coordinate parameters given, and we are done here')
      print('\n')


    else:
      print('Nothing to do, moving on, have a nice day')  
      print('\n') 


  #All exps      
    all_exps_corp = pd.merge(all_exps, all_piv_corp, left_on = ['business_unit', 'spend_category_as_worktag','supplier_as_worktag'], right_on =['business_unit', 'spend_category_as_worktag','supplier_as_worktag'], how = 'outer').replace(np.nan,"",regex = True)
    all_exps_corp = all_exps_corp[(all_exps_corp['business_unit']=='CNET_group_corp')]  


    print('Do you want to update the CNET Corp sheet with ALL EXPENSE data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What all expense row #(only numerical values)?')
      row = int(input())
      print('What all expense column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_corp_ws,all_exps_corp,row,column)
      print('new data has been populated in coordinate parameters given')
      print('\n')
  
    elif(response == 'Yes'):
      print('What all expense row #(only numerical values)?')
      row = int(input())
      print('What all expense column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(cnet_corp_ws,all_exps_corp,row,column)
      print('new data has been populated in coordinate parameters given, and we are done here')
      print('\n')


    else:
      print('Nothing to do, moving on, have a nice day')  
      print('\n')        

## ZD Net

In [ ]:
for df in updated_bus_units:
  if id(df) in [id(B753_B2B___ZDNet)]:



    #new expense df for the passed business units
    #exp = df[(df['expense_flag'] == 'm&c_expenses')]
    bu_piv_zd = bu_piv[(bu_piv['business_unit']=='B753_B2B___ZDNet')]
    spend_piv_zd = spend_piv[(spend_piv['business_unit']=='B753_B2B___ZDNet')]
    supp_piv_zd = supp_piv[(supp_piv['business_unit']=='B753_B2B___ZDNet')]
    all_piv_zd = all_piv[(all_piv['business_unit']=='B753_B2B___ZDNet')] 
       
     # Business Unit
    bus_exps_zd = pd.merge(bus_exps, bu_piv_zd, left_on = 'business_unit', right_on ='business_unit', how = 'right').replace(np.nan,"",regex = True)
    bus_exps_zd = bus_exps_zd[(bus_exps_zd['business_unit']=='B753_B2B___ZDNet')]

    print('Do you want to update the ZD Net sheet with new BUSINESS UNIT data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What business unit row #(only numerical values)?')
      row = int(input())
      print('What business unit column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(zdnet_ws,bus_exps_zd,row,column)
      print('new data within the ZD Net tab has been populated with the coordinate parameters given')
      print('\n')
  
    elif(response == 'Yes'):
      print('What business unit row #(only numerical values)?')
      row = int(input())
      print('What business unit column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(zdnet_ws,bus_exps_zd,row,column)
      print('new data within the ZD Net tab has been populated with the coordinate parameters given')
      print('\n')

    else:
      print('Nothing to do, moving on, have a nice day')
      print('\n')

  #spend category
    spend_exps_zd = pd.merge(spend_exps, spend_piv_zd, left_on = ['business_unit','spend_category_as_worktag'], right_on = ['business_unit','spend_category_as_worktag'], how = 'outer').replace(np.nan,"",regex = True)
    spend_exps_zd = spend_exps_zd[(spend_exps_zd['business_unit']=='B753_B2B___ZDNet')]

    print('Do you want to update the ZD Net sheet with new SPEND CATEGORY data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What spend category row #(only numerical values)?')
      row = int(input())
      print('What spend category column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(zdnet_ws,spend_exps_zd,row,column)
      print('new data has been populated in the ZD Net tab with the coordinate parameters given')
      print('\n')
  
    elif(response == 'Yes'):
      print('What spend category row #(only numerical values)?')
      row = int(input())
      print('What column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(zdnet_ws,spend_exps_zd,row,column)
      print('new data has been populated in the ZD Net tab with the coordinate parameters given')
      print('\n')

    else:
      print('Nothing to do, moving on, have a nice day')
      print('\n')

  #Supplier As Worktag
    #supp_exps =pd.pivot_table(exp, index = ['business_unit','supplier_as_worktag'], columns = ['month_dt'], values = 'workday_rev_actuals', aggfunc= np.sum, fill_value = 0).reset_index()  
    supp_exps_zd = pd.merge(supp_exps, supp_piv_zd, left_on = ['business_unit', 'supplier_as_worktag'], right_on =['business_unit','supplier_as_worktag'], how = 'outer').replace(np.nan,"",regex = True)
    supp_exps_zd = supp_exps_zd[(supp_exps_zd['business_unit']=='B753_B2B___ZDNet')]


    print('Do you want to update the ZD Net sheet with SUPPLIER SPEND data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What supplier spend row #(only numerical values)?')
      row = int(input())
      print('What supplier spend column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(zdnet_ws,supp_exps_zd,row,column)
      print('new data has been populated in the ZD Net tab with the coordinate parameters given, and we are done here')
      print('\n')
  
    elif(response == 'Yes'):
      print('What supplier spend row #(only numerical values)?')
      row = int(input())
      print('What supplier spend column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(zdnet_ws,supp_exps_zd,row,column)
      print('new data has been populated in the ZD Net tab with the coordinate parameters given, and we are done here')
      print('\n')


    else:
      print('Nothing to do, moving on, have a nice day')  
      print('\n') 


  #All exps      
    all_exps_zd = pd.merge(all_exps, all_piv_zd, left_on = ['business_unit', 'spend_category_as_worktag','supplier_as_worktag'], right_on =['business_unit', 'spend_category_as_worktag','supplier_as_worktag'], how = 'outer').replace(np.nan,"",regex = True)
    all_exps_zd = all_exps_zd[(all_exps_zd['business_unit']=='B753_B2B___ZDNet')]  


    print('Do you want to update the ZD Net sheet with ALL EXPENSE data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What all expense row #(only numerical values)?')
      row = int(input())
      print('What all expense column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(zdnet_ws,all_exps_zd,row,column)
      print('new data has been populated in coordinate parameters given')
      print('\n')
  
    elif(response == 'Yes'):
      print('What all expense row #(only numerical values)?')
      row = int(input())
      print('What all expense column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(zdnet_ws,all_exps_zd,row,column)
      print('new data has been populated in coordinate parameters given, and we are done here')
      print('\n')


    else:
      print('Nothing to do, moving on, have a nice day')  
      print('\n')      

## Time

In [ ]:
for df in updated_bus_units:
  if id(df) in [id(B548_TIME)]:



    #new expense df for the passed business units
    #exp = df[(df['expense_flag'] == 'm&c_expenses')]
    bu_piv_time = bu_piv[(bu_piv['business_unit']=='B548_TIME')]
    spend_piv_time = spend_piv[(spend_piv['business_unit']=='B548_TIME')]
    supp_piv_time = supp_piv[(supp_piv['business_unit']=='B548_TIME')]
    all_piv_time = all_piv[(all_piv['business_unit']=='B548_TIME')] 
       
     # Business Unit
    bus_exps_time = pd.merge(bus_exps, bu_piv_time, left_on = 'business_unit', right_on ='business_unit', how = 'right').replace(np.nan,"",regex = True)
    bus_exps_time = bus_exps_time[(bus_exps_time['business_unit']=='B548_TIME')]

    print('Do you want to update the TIME sheet with new BUSINESS UNIT data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What business unit row #(only numerical values)?')
      row = int(input())
      print('What business unit column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(time_ws,bus_exps_time,row,column)
      print('new data within the Time tab has been populated with the coordinate parameters given')
      print('\n')
  
    elif(response == 'Yes'):
      print('What business unit row #(only numerical values)?')
      row = int(input())
      print('What business unit column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(time_ws,bus_exps_time,row,column)
      print('new data within the Time tab has been populated with the coordinate parameters given')
      print('\n')

    else:
      print('Nothing to do, moving on, have a nice day')
      print('\n')

  #spend category
    spend_exps_time = pd.merge(spend_exps, spend_piv_time, left_on = ['business_unit','spend_category_as_worktag'], right_on = ['business_unit','spend_category_as_worktag'], how = 'outer').replace(np.nan,"",regex = True)
    spend_exps_time = spend_exps_time[(spend_exps_time['business_unit']=='B548_TIME')]

    print('Do you want to update the Time sheet with new SPEND CATEGORY data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What spend category row #(only numerical values)?')
      row = int(input())
      print('What spend category column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(time_ws,spend_exps_time,row,column)
      print('new data has been populated in the Time tab with the coordinate parameters given')
      print('\n')
  
    elif(response == 'Yes'):
      print('What spend category row #(only numerical values)?')
      row = int(input())
      print('What column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(time_ws,spend_exps_time,row,column)
      print('new data has been populated in the Time tab with the coordinate parameters given')
      print('\n')

    else:
      print('Nothing to do, moving on, have a nice day')
      print('\n')

  #Supplier As Worktag
    #supp_exps =pd.pivot_table(exp, index = ['business_unit','supplier_as_worktag'], columns = ['month_dt'], values = 'workday_rev_actuals', aggfunc= np.sum, fill_value = 0).reset_index()  
    supp_exps_time = pd.merge(supp_exps, supp_piv_time, left_on = ['business_unit', 'supplier_as_worktag'], right_on =['business_unit','supplier_as_worktag'], how = 'outer').replace(np.nan,"",regex = True)
    supp_exps_time = supp_exps_time[(supp_exps_time['business_unit']=='B548_TIME')]


    print('Do you want to update the Time sheet with SUPPLIER SPEND data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What supplier spend row #(only numerical values)?')
      row = int(input())
      print('What supplier spend column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(time_ws,supp_exps_time,row,column)
      print('new data has been populated in the Time tab with the coordinate parameters given, and we are done here')
      print('\n')
  
    elif(response == 'Yes'):
      print('What supplier spend row #(only numerical values)?')
      row = int(input())
      print('What supplier spend column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(time_ws,supp_exps_time,row,column)
      print('new data has been populated in the Time tab with the coordinate parameters given, and we are done here')
      print('\n')


    else:
      print('Nothing to do, moving on, have a nice day')  
      print('\n') 


  #All exps      
    all_exps_time = pd.merge(all_exps, all_piv_time, left_on = ['business_unit', 'spend_category_as_worktag','supplier_as_worktag'], right_on =['business_unit', 'spend_category_as_worktag','supplier_as_worktag'], how = 'outer').replace(np.nan,"",regex = True)
    all_exps_time = all_exps_time[(all_exps_time['business_unit']=='B548_TIME')]  


    print('Do you want to update the Time sheet with ALL EXPENSE data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What all expense row #(only numerical values)?')
      row = int(input())
      print('What all expense column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(time_ws,all_exps_time,row,column)
      print('new data has been populated in coordinate parameters given')
      print('\n')
  
    elif(response == 'Yes'):
      print('What all expense row #(only numerical values)?')
      row = int(input())
      print('What all expense column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(time_ws,all_exps_time,row,column)
      print('new data has been populated in coordinate parameters given, and we are done here')
      print('\n')


    else:
      print('Nothing to do, moving on, have a nice day')  
      print('\n')       

## My Slumber Yard

In [ ]:
for df in updated_bus_units:
  if id(df) in [id(B199_MySlumberYard_com)]:



    #new expense df for the passed business units
    #exp = df[(df['expense_flag'] == 'm&c_expenses')]
    bu_piv_msy = bu_piv[(bu_piv['business_unit']=='B199_MySlumberYard_com')]
    spend_piv_msy = spend_piv[(spend_piv['business_unit']=='B199_MySlumberYard_com')]
    supp_piv_msy = supp_piv[(supp_piv['business_unit']=='B199_MySlumberYard_com')]
    all_piv_msy = all_piv[(all_piv['business_unit']=='B199_MySlumberYard_com')] 
       
     # Business Unit
    bus_exps_msy = pd.merge(bus_exps, bu_piv_msy, left_on = 'business_unit', right_on ='business_unit', how = 'right').replace(np.nan,"",regex = True)
    bus_exps_msy = bus_exps_msy[(bus_exps_msy['business_unit']=='B199_MySlumberYard_com')]

    print('Do you want to update the MSY sheet with new BUSINESS UNIT data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What business unit row #(only numerical values)?')
      row = int(input())
      print('What business unit column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(msy_ws,bus_exps_msy,row,column)
      print('new data within the MSY tab has been populated with the coordinate parameters given')
      print('\n')
  
    elif(response == 'Yes'):
      print('What business unit row #(only numerical values)?')
      row = int(input())
      print('What business unit column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(msy_ws,bus_exps_msy,row,column)
      print('new data within the MSY tab has been populated with the coordinate parameters given')
      print('\n')

    else:
      print('Nothing to do, moving on, have a nice day')
      print('\n')

  #spend category
    spend_exps_msy = pd.merge(spend_exps, spend_piv_msy, left_on = ['business_unit','spend_category_as_worktag'], right_on = ['business_unit','spend_category_as_worktag'], how = 'outer').replace(np.nan,"",regex = True)
    spend_exps_msy = spend_exps_msy[(spend_exps_msy['business_unit']=='B199_MySlumberYard_com')]

    print('Do you want to update the MSY sheet with new SPEND CATEGORY data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What spend category row #(only numerical values)?')
      row = int(input())
      print('What spend category column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(msy_ws,spend_exps_msy,row,column)
      print('new data has been populated in the MSY tab with the coordinate parameters given')
      print('\n')
  
    elif(response == 'Yes'):
      print('What spend category row #(only numerical values)?')
      row = int(input())
      print('What column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(msy_ws,spend_exps_msy,row,column)
      print('new data has been populated in the MSY tab with the coordinate parameters given')
      print('\n')

    else:
      print('Nothing to do, moving on, have a nice day')
      print('\n')

  #Supplier As Worktag
    #supp_exps =pd.pivot_table(exp, index = ['business_unit','supplier_as_worktag'], columns = ['month_dt'], values = 'workday_rev_actuals', aggfunc= np.sum, fill_value = 0).reset_index()  
    supp_exps_msy = pd.merge(supp_exps, supp_piv_msy, left_on = ['business_unit', 'supplier_as_worktag'], right_on =['business_unit','supplier_as_worktag'], how = 'outer').replace(np.nan,"",regex = True)
    supp_exps_msy = supp_exps_msy[(supp_exps_msy['business_unit']=='B199_MySlumberYard_com')]


    print('Do you want to update the MSY sheet with SUPPLIER SPEND data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What supplier spend row #(only numerical values)?')
      row = int(input())
      print('What supplier spend column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(msy_ws,supp_exps_msy,row,column)
      print('new data has been populated in the MSY tab with the coordinate parameters given, and we are done here')
      print('\n')
  
    elif(response == 'Yes'):
      print('What supplier spend row #(only numerical values)?')
      row = int(input())
      print('What supplier spend column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(msy_ws,supp_exps_msy,row,column)
      print('new data has been populated in the MSY tab with the coordinate parameters given, and we are done here')
      print('\n')


    else:
      print('Nothing to do, moving on, have a nice day')  
      print('\n') 

  #All exps      
    all_exps_msy = pd.merge(all_exps, all_piv_msy, left_on = ['business_unit', 'spend_category_as_worktag','supplier_as_worktag'], right_on =['business_unit', 'spend_category_as_worktag','supplier_as_worktag'], how = 'outer').replace(np.nan,"",regex = True)
    all_exps_msy = all_exps_msy[(all_exps_msy['business_unit']=='B199_MySlumberYard_com')]  


    print('Do you want to update the MSY sheet with ALL EXPENSE data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What all expense row #(only numerical values)?')
      row = int(input())
      print('What all expense column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(msy_ws,all_exps_msy,row,column)
      print('new data has been populated in coordinate parameters given')
      print('\n')
  
    elif(response == 'Yes'):
      print('What all expense row #(only numerical values)?')
      row = int(input())
      print('What all expense column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(msy_ws,all_exps_msy,row,column)
      print('new data has been populated in coordinate parameters given, and we are done here')
      print('\n')


    else:
      print('Nothing to do, moving on, have a nice day')  
      print('\n') 


## Product

In [ ]:
for df in updated_bus_units:
  if id(df) in [id(B930_Product)]:



    #new expense df for the passed business units
    #exp = df[(df['expense_flag'] == 'm&c_expenses')]
    bu_piv_prod = bu_piv[(bu_piv['business_unit']=='B930_Product')]
    spend_piv_prod = spend_piv[(spend_piv['business_unit']=='B930_Product')]
    supp_piv_prod = supp_piv[(supp_piv['business_unit']=='B930_Product')]
    all_piv_prod = all_piv[(all_piv['business_unit']=='B930_Product')] 
       
     # Business Unit
    bus_exps_prod = pd.merge(bus_exps, bu_piv_prod, left_on = 'business_unit', right_on ='business_unit', how = 'right').replace(np.nan,"",regex = True)
    bus_exps_prod = bus_exps_prod[(bus_exps_prod['business_unit']=='B930_Product')]

    print('Do you want to update the Prod sheet with new BUSINESS UNIT data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What business unit row #(only numerical values)?')
      row = int(input())
      print('What business unit column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(product_ws,bus_exps_prod,row,column)
      print('new data within the Prod tab has been populated with the coordinate parameters given')
      print('\n')
  
    elif(response == 'Yes'):
      print('What business unit row #(only numerical values)?')
      row = int(input())
      print('What business unit column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(product_ws,bus_exps_prod,row,column)
      print('new data within the MSY tab has been populated with the coordinate parameters given')
      print('\n')

    else:
      print('Nothing to do, moving on, have a nice day')
      print('\n')

  #spend category
    spend_exps_prod = pd.merge(spend_exps, spend_piv_prod, left_on = ['business_unit','spend_category_as_worktag'], right_on = ['business_unit','spend_category_as_worktag'], how = 'outer').replace(np.nan,"",regex = True)
    spend_exps_prod = spend_exps_prod[(spend_exps_prod['business_unit']=='B930_Product')]

    print('Do you want to update the Prod sheet with new SPEND CATEGORY data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What spend category row #(only numerical values)?')
      row = int(input())
      print('What spend category column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(product_ws,spend_exps_prod,row,column)
      print('new data has been populated in the Prod tab with the coordinate parameters given')
      print('\n')
  
    elif(response == 'Yes'):
      print('What spend category row #(only numerical values)?')
      row = int(input())
      print('What column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(product_ws,spend_exps_prod,row,column)
      print('new data has been populated in the Prod tab with the coordinate parameters given')
      print('\n')

    else:
      print('Nothing to do, moving on, have a nice day')
      print('\n')

  #Supplier As Worktag
    #supp_exps =pd.pivot_table(exp, index = ['business_unit','supplier_as_worktag'], columns = ['month_dt'], values = 'workday_rev_actuals', aggfunc= np.sum, fill_value = 0).reset_index()  
    supp_exps_prod = pd.merge(supp_exps, supp_piv_prod, left_on = ['business_unit', 'supplier_as_worktag'], right_on =['business_unit','supplier_as_worktag'], how = 'outer').replace(np.nan,"",regex = True)
    supp_exps_prod = supp_exps_prod[(supp_exps_prod['business_unit']=='B930_Product')]


    print('Do you want to update the MSY sheet with SUPPLIER SPEND data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What supplier spend row #(only numerical values)?')
      row = int(input())
      print('What supplier spend column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(product_ws,supp_exps_prod,row,column)
      print('new data has been populated in the MSY tab with the coordinate parameters given, and we are done here')
      print('\n')
  
    elif(response == 'Yes'):
      print('What supplier spend row #(only numerical values)?')
      row = int(input())
      print('What supplier spend column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(product_ws,supp_exps_prod,row,column)
      print('new data has been populated in the MSY tab with the coordinate parameters given, and we are done here')
      print('\n')


    else:
      print('Nothing to do, moving on, have a nice day')  
      print('\n') 

  #All exps      
    all_exps_prod = pd.merge(all_exps, all_piv_prod, left_on = ['business_unit', 'spend_category_as_worktag','supplier_as_worktag'], right_on =['business_unit', 'spend_category_as_worktag','supplier_as_worktag'], how = 'outer').replace(np.nan,"",regex = True)
    all_exps_prod = all_exps_prod[(all_exps_prod['business_unit']=='B930_Product')]  


    print('Do you want to update the Product sheet with ALL EXPENSE data?, yes/no')
    response = input()
    if(response == 'yes'):
      print('What all expense row #(only numerical values)?')
      row = int(input())
      print('What all expense column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(product_ws,all_exps_prod,row,column)
      print('new data has been populated in coordinate parameters given')
      print('\n')
  
    elif(response == 'Yes'):
      print('What all expense row #(only numerical values)?')
      row = int(input())
      print('What all expense column #(only numerical values)?')
      column = int(input(_))
      set_with_dataframe(product_ws,all_exps_prod,row,column)
      print('new data has been populated in coordinate parameters given, and we are done here')
      print('\n')


    else:
      print('Nothing to do, moving on, have a nice day')  
      print('\n')       

In [ ]:
xx

In [ ]:
#@ original bu pivot to sheets
# Business Unit (original)




bu_piv = nhc_budg.worksheet('bu_non_hc_budget_pivot').get('A1:M8')
# change budget month name per month
bu_piv = df_filter(df_format(bu_piv),'business_unit','Jan_budg')
bus_exps = pd.pivot_table(exp, index = ['business_unit'], columns = ['month_dt'], values = 'workday_rev_actuals', aggfunc= np.sum, fill_value = 0).reset_index()
bus_exps = pd.merge(bus_exps, bu_piv, left_on = 'business_unit', right_on ='business_unit', how = 'right').replace(np.nan,"",regex = True)
bu_wksht = nhc_budg.worksheet('bu_non_hc_budget_pivot')
print('Do you want to update google sheets with new data?, yes/no')
response = input()
if(response == 'yes'):
  print('What row #(only numberical values)?')
  row = int(input())
  print('What column #(only numberical values)?')
  column = int(input(_))
  set_with_dataframe(bu_wksht,bus_exps,row,column)
  print('new data has been populated in coordinate parameters given')
  
elif(response == 'Yes'):
  print('What row #(only numberical values)?')
  row = int(input())
  print('What column #(only numberical values)?')
  column = int(input(_))
  set_with_dataframe(bu_wksht,bus_exps,row,column)
  print('new data has been populated in coordinate parameters given')


else:
  print('Nothing to do, moving on, have a nice day')

In [ ]:
yes#spend category as worktag original
spend_piv = nhc_budg.worksheet('spend_cat_non_hc_budget_pivot').get('A1:M71')
# change budget month name per month
spend_piv = df_filter(df_format(spend_piv),'spend_category_as_worktag','Jan_budg')
spend_exps = pd.pivot_table(exp, index = ['spend_category_as_worktag'], columns = ['month_dt'], values = 'workday_rev_actuals', aggfunc= np.sum, fill_value = 0).reset_index()
pd.set_option('display.max_rows', 110)
spend_exps = pd.merge(spend_exps, spend_piv, left_on = 'spend_category_as_worktag', right_on ='spend_category_as_worktag', how = 'outer').replace(np.nan,"",regex = True)

spend_wksht = nhc_budg.worksheet('spend_cat_non_hc_budget_pivot')


print('Do you want to update google sheets with new data?, yes/no')
response = input()
if(response == 'yes'):
  print('What row #(only numberical values)?')
  row = int(input())
  print('What column #(only numberical values)?')
  column = int(input(_))
  set_with_dataframe(spend_wksht,spend_exps,row,column)
  print('new data has been populated in coordinate parameters given')
  
elif(response == 'Yes'):
  print('What row #(only numberical values)?')
  row = int(input())
  print('What column #(only numberical values)?')
  column = int(input(_))
  set_with_dataframe(spend_wksht,spend_exps,row,column)
  print('new data has been populated in coordinate parameters given')


else:
  print('Nothing to do, moving on, have a nice day')


In [ ]:
1y#Supplier As Worktag

supp_piv = nhc_budg.worksheet('supplier_non_hc_budget_pivot').get('A1:N760')
# change budget month name per month
supp_piv = df_filter(df_format(supp_piv),"supplier_as_worktag","Jan_budg")
supp_exps =pd.pivot_table(exp, index = ['supplier_as_worktag'], columns = ['month_dt'], values = 'workday_rev_actuals', aggfunc= np.sum, fill_value = 0).reset_index()
supp_exps = pd.merge(supp_exps, supp_piv, left_on = 'supplier_as_worktag', right_on ='supplier_as_worktag', how = 'outer').replace(np.nan,"",regex = True)


supp_wksht = nhc_budg.worksheet('supplier_comp_sheet')


print('Do you want to update google sheets with new data?, yes/no')
response = input()
if(response == 'yes'):
  print('What row #(only numberical values)?')
  row = int(input())
  print('What column #(only numberical values)?')
  column = int(input(_))
  set_with_dataframe(supp_wksht,supp_exps,row,column)
  print('new data has been populated in coordinate parameters given')
  
elif(response == 'Yes'):
  print('What row #(only numberical values)?')
  row = int(input())
  print('What column #(only numberical values)?')
  column = int(input(_))
  set_with_dataframe(supp_wksht,supp_exps,row,column)
  print('new data has been populated in coordinate parameters given')


else:
  print('Nothing to do, moving on, have a nice day')